Main notebook for analysis and plot-making for the project

In [43]:
import re
from collections import defaultdict
from itertools import product
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from matplotlib.ticker import FixedLocator, LogLocator, MultipleLocator
from my_favorite_things import (
    format_ddict,
    histbar,
    multifader,
    nested_ddict,
    pprint_nested_dict,
)
from numba import njit
from scripts.constants import (
    ALG_CHOICES,
    DATA_CHOICES,
    DATA_DIR,
    EVENT_CHOICES,
    EVT_DIR,
    INVMS,
    METRIC,
    NUM_FSP_DICT,
    OUTPUT_DIR,
    PLOTS_DIR,
    POSTDATA_DIR,
    QCL_CHOICES,
    SYM_TRUE_BS_DICT,
    TOP_MASS,
    W_MASS,
)
from scripts.qc_utilities import format_p4s, get_info, get_Jijs_Pijs, swap

In [3]:
plt.rcParams.update(
    {
        "axes.labelsize": 24,
        "ytick.labelsize": 18,
        "xtick.labelsize": 18,
        "figure.subplot.wspace": 0.25,
        "figure.subplot.hspace": 0.2,
        "axes.titlesize": 26,
        "legend.fontsize": 15,
    }
)

# Constants

In [4]:
# Random State object for reproducibility
RNG = np.random.RandomState(100)
# Folders for postdata
ALG_DATA = POSTDATA_DIR / "algorithms"
BF_DATA = POSTDATA_DIR / "hamiltonian"
SHOT_DATA = DATA_DIR / "shot_data"

&nbsp;<hr style="border:5px solid #8d99ae">
# Find Used $p$'s
Print out the depths simulated for each choice of event type, data type, Hamiltonian and algorithm. FALQON has the data for every depth _up to_ the shown depth.

In [5]:
depth_dict = nested_ddict(3, list)
for path in OUTPUT_DIR.glob("*/*"):
    # print(path)
    etype, dtype, alg, quadcoeff, depth = get_info(path, ["etype", "dtype", "alg", "quadcoeff", "depth"])

    if int(depth) not in depth_dict[etype][dtype][quadcoeff][alg]:
        depth_dict[etype][dtype][quadcoeff][alg].append(int(depth))

depth_dict = format_ddict(depth_dict)

In [ ]:
pprint_nested_dict(depth_dict, v_format=lambda varr: sorted(varr))

&nbsp;<hr style="border:5px solid #8d99ae">
# Plot Efficiency vs Invariant Mass

In [7]:
# line styles
LSS = ["solid", "dashed", "dotted", "dashdot"]
# colors
col_lims = ["#ffba08", "#f48c06", "#dc2f02", "#9d0208", "#370617"]
# COLORS = multifader(col_lims, [0, 0.33, 0.66, 1])
COLORS = ["#FFBA08", "#F48C06", "#DC2F02", "#5C2751", "#39A0ED"]
# alpha
ALPHA = 1
# line width
LW = 3

# Define what maximum to plot
max_params = [["ttbar", "parton", "H0"], ["gray", "-", 0.6]]
# Define what data for the Hemisphere method to use
hemisphere_params = [["ttbar", "parton"], ["k", "dashdot", 1, 3]]
# Define what lines to plot
data_params = np.array(
    [
        ["ttbar", "parton", "H0", "qaoa", 8],
        ["ttbar", "parton", "H0", "maqaoa", 8],
        ["ttbar", "parton", "H0", "falqon", 200],
        # ["ttbar", "parton", "H0", "maqaoa", 3],
        # ["ttbar", "parton", "H0", "maqaoa", 3, True, 0.1],
        # ["ttbar", "parton", "H0", "maqaoa", 3, True, 1],
    ],
    dtype=object,
)
# Define their params: color, linestyle, alpha, linewidth
plot_params = [
    [COLORS[0], LSS[0], ALPHA, LW],
    [COLORS[1], LSS[1], ALPHA, LW],
    [COLORS[2], LSS[2], ALPHA, LW],
    # [COLORS[4], LSS[0], ALPHA, LW],
    # [COLORS[4], LSS[1], ALPHA, LW],
    # [COLORS[4], LSS[2], ALPHA, LW],
]


# Define what patches to show on the legend: color, label
patches_params = [
    # [COLORS[0], "$p=2$"],
    # [COLORS[1], "$p=3$"],
    # [COLORS[2], "$p=4$"],
    # [COLORS[3], "$p=5$"],
    # [COLORS[4], "$p=4$"]
]
# Define what lines to show on the legend: linestyle, color, label
lines_params = [
    [COLORS[0], LSS[0], ALPHA, LW, "QAOA\n[$p=8$]"],
    [COLORS[1], LSS[1], ALPHA, LW, "maQAOA\n[$p=8$]"],
    [COLORS[2], LSS[2], ALPHA, LW, "FALQON\n[$p=250$]"],
    ["k", LSS[3], ALPHA, LW, "Hemisphere\nmethod"],
    # [COLORS[4], LSS[0], ALPHA, LW, "Ideal"],
    # [COLORS[4], LSS[1], ALPHA, LW, "0.1% Bitflip"],
    # [COLORS[4], LSS[2], ALPHA, LW, "1% Bitflip"],
]
# An extra legend of stuff
extras_params = []
legend_label = ""  # "ma-QAOA [$p=3$]"

In [ ]:
def eff_plot(
    data_params,
    plot_params,
    lines_params,
    patches_params,
    max_params,
    hemisphere_params,
    legend_label="",
    extras_params=[],
    correct_or_min="correct",
    legend=True,
):
    """
    Plots efficiency vs invariant mass. And automatically takes care of legend and such.

    Parameters:
    data_params - Parameters of data to plot as
            [<event type>, <data type>, <Hamiltonian>, <algorithm>, <depth>]
        with an optional [<is noise>, <noise prob>] for noisy simulations. For example,
            ["ttbar", "parton", "H0", "maqaoa", 8]
    plot_params - Parameters for the plotted lines as
            [<color>, <line style>, <alpha>, <line width>]
    lines_params - Parameters for the lines in the legend as
            [<color>, <line style>, <alpha>, <line width>, <label>]
    patches_params - Parameters for the patches in the legend as
            [<color>, <label>]
    max_params - Parameters for filled-in maximum as
            [[<event type>, <data type>, <Hamiltonian>], [<color>, <hatch>, <alpha>]]
    hemisphere_params - Parameters for hemisphere method as
            [[<event type>, <data type>], [<color>, <line style>, <alpha>, <line width>]]
    legend_label (default "") - The title for the legend
    extras_params (default []) - Parameters for any extra lines to add to legend as
            [<color>, <line style>, <alpha>, <line width>, <label>]
    correct_or_min (default "correct") - Whether to plot the efficiency of finding the
        minimum energy ("min") or the correct bit string ("correct"). If "min", then
        "max_params" will not be used.
    legend (default True) - If True, will show the legend, otherwise it will not.
    """
    # Load in algorithm data
    plot_data = []
    for datum_params in data_params:
        if len(datum_params) == 5:
            # Without noise
            etype, dtype, qcl, alg, p = datum_params
            quadcoeff = qcl.split("_")[0]
            data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz")
        else:
            # With noise
            etype, dtype, qcl, alg, p, noise, noise_prob = datum_params
            quadcoeff = qcl.split("_")[0]
            noisy = f"noisy{noise_prob}_"
            data = np.load(ALG_DATA / f"{noisy}{alg}_{etype}_{dtype}_{quadcoeff}.npz")

        if alg == "falqon":
            effs = data[f"{correct_or_min}_effs"][int(p)]
        else:
            effs = data[f"{correct_or_min}_effs_p{p}"]
        plot_data.append(effs)

    # Load in hemisphere data
    if hemisphere_params:
        hemisphere_effs = []
        for invm in INVMS[:-1]:
            fname = f"hemisphere_{hemisphere_params[0][0]}_{hemisphere_params[0][1]}.npz"
            data = np.load(BF_DATA / f"indexed_{invm:.2f}" / fname)
            hemisphere_effs.append(float(data["eff"]))

    # Figure and axis
    fig, ax = plt.subplots(figsize=(10, 10))

    # Plotting max, shaded region
    max_handles = []
    if max_params and correct_or_min == "correct":
        max_etype, max_dtype, max_qcl = max_params[0]
        max_data = [np.load(BF_DATA / f"indexed_{invm:.2f}" / f"effs_{max_etype}_{max_dtype}.npz")[max_qcl][0] for invm in INVMS[:-1]]
        color, hatch, alpha = max_params[1]

        histbar(
            ax,
            xs=INVMS,
            ys=max_data,
            capends=False,
            label_type="explicit",
            color=color,
            fill=True,
            fill_limit=1,
            fill_alpha=alpha,
            fill_color=color,
            fill_hatch=hatch,
        )
        max_handles = [
            # Patch(color=color, alpha=alpha, hatch=hatch, label="Hamiltonian\nLlimit")
        ]

    # Plotting algorithm data
    for plot_datum, plot_param in zip(plot_data, plot_params):
        color, ls, alpha, lw = plot_param
        histbar(
            ax,
            xs=INVMS,
            ys=plot_datum,
            capends=False,
            label_type="explicit",
            color=color,
            ls=ls,
            alpha=alpha,
            lw=lw,
        )

    # Plotting hemisphere algorithm data
    hemisphere_handle = []
    if hemisphere_params:
        color, ls, alpha, lw = hemisphere_params[1]
        label = "Hemisphere\nmethod"
        # Plotting hemisphere method
        histbar(
            ax,
            xs=INVMS,
            ys=hemisphere_effs,
            capends=False,
            label_type="explicit",
            color=color,
            ls=ls,
            lw=lw,
            alpha=alpha,
        )
        # hemisphere_handle.append(
        #     Line2D([0], [0], color=color, ls=ls, alpha=alpha, lw=lw, label=label)
        # )

    # Create handles for the legend
    line_handles = []
    for line_params in lines_params:
        color, ls, alpha, lw, label = line_params
        line = Line2D([0], [0], color=color, ls=ls, alpha=alpha, lw=lw, label=label)
        line_handles.append(line)
    extra_handles = []
    for extra_params in extras_params:
        color, ls, alpha, lw, label = extra_params
        handle = Line2D([0], [0], color=color, ls=ls, alpha=alpha, lw=lw, label=label)
        extra_handles.append(handle)
    patch_handles = []
    for patch_params in patches_params:
        color, label = patch_params
        patch_handles.append(Patch(color=color, label=label))

    # Axis stuff
    ax.set_xlim(INVMS[0], INVMS[-1])
    ax.set_ylim(0, 1)
    ax.tick_params(top=True, right=True, which="both")
    ax.tick_params(axis="both")
    ax.yaxis.set_major_locator(MultipleLocator(0.25))
    ax.yaxis.set_minor_locator(MultipleLocator(0.05))
    ax.grid(which="major", alpha=0.3)

    # Legend
    spacer = []  # [Patch(alpha=0)]
    handles = max_handles + patch_handles + spacer + hemisphere_handle + line_handles

    if extra_handles:
        handles += spacer + extra_handles
    if legend:
        fig.legend(
            title=legend_label,
            handles=handles,
            bbox_to_anchor=[1.13, 0.5],
            loc=5,
            fontsize=13,
            edgecolor="k",
            shadow=True,
            fancybox=True,
            labelspacing=1,
            handlelength=4,
            handleheight=3,
            title_fontsize=15,
            # ncol=7,
        )

    # Finds label for x-axis based on event type
    if np.unique([etype_param[0] for etype_param in data_params]).shape == (1,):
        xlabel = {
            "ttbar": "m_{tt}/(2m_t)",
            "tW": "m_{tW}/(m_t+m_W)",
            "6jet": r"m_{\rm jets}/(2m_t)",
        }[data_params[0][0]]
    else:
        xlabel = "m_{AB}/(m_A+m_B)"
    ax.set_xlabel(f"${xlabel}$")
    ax.set_ylabel("Efficiency")
    ax.set_box_aspect(1)

    return fig, ax


fig, ax = eff_plot(
    data_params=data_params,
    plot_params=plot_params,
    hemisphere_params=hemisphere_params,
    max_params=max_params,
    lines_params=lines_params,
    patches_params=patches_params,
    extras_params=extras_params,
    legend_label=legend_label,
    correct_or_min="correct",
    legend=True,
)

# plt.savefig(PLOTS_DIR / "misc" / "eff_noisy_p8.png", bbox_inches="tight")
# plt.savefig(PLOTS_DIR / "misc" / "eff_noisy_0.1.png", bbox_inches="tight")

&nbsp;<hr style="border:5px solid #8d99ae">
# Mass 2D Histograms and Jet Number Histograms

In [9]:
def mass_histogram(ax, masses1, masses2, etype, lims=(50, 500), locs=(50, 100), bins=250):
    """
    Create a 2D histogram of `masses1` vs `masses2` with a log scale colorbar.

    Parameters:
    ax - Matplotlib axis to plot histogram on
    masses1 - Array of masses for particle 1
    masses2 - Array of masses for particle 2
    etype - Event type, e.g. "parton" or "smeared"
    lims (default (50, 500)) - The limits on the x and y axes
    locs (default (50, 100)) - Span of major and minor ticks on x and y axes
    bins (default 250) - Number of bins
    """
    # How to label x and y axes
    labels = {"ttbar": ["m_t", "m_t"], "tW": ["m_t", "m_W"], "6jet": ["m_1", "m_2"]}[etype]
    # Correct masses to label with dashed lines
    exact_masses = {
        "ttbar": [TOP_MASS, TOP_MASS],
        "tW": [TOP_MASS, W_MASS],
        "6jet": [0, 0],
    }[etype]

    # Plot data
    ax.hist2d(masses1, masses2, bins=bins, norm=LogNorm(), density=True, cmap="RdPu")
    ax.axvline(exact_masses[0], c="k", ls=":", alpha=0.4)
    ax.axhline(exact_masses[1], c="k", ls=":", alpha=0.4)
    # x-axis stuff
    ax.set_xlim(lims)
    ax.set_xlabel(f"${labels[0]}$ (GeV)")
    ax.xaxis.set_major_locator(MultipleLocator(locs[0]))
    ax.xaxis.set_minor_locator(MultipleLocator(locs[1]))
    # y-axis stuff
    ax.set_ylim(lims)
    ax.set_ylabel(f"${labels[1]}$ (GeV)")
    ax.yaxis.set_major_locator(MultipleLocator(locs[0]))
    ax.yaxis.set_minor_locator(MultipleLocator(locs[1]))

    ax.grid(which="major", alpha=0.4)
    ax.grid(which="minor", alpha=0.2)

    ax.set_axisbelow(True)
    # ax.set_box_aspect(1)


def jet_histogram(ax, data, max_jets):
    bar = ax.bar(range(len(data)), data / sum(data), color="#7B3E19")
    ax.set_xlim(-0.5, max_jets + 0.5)
    ax.set_xlabel("Number of jets")

    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    ax.set_ylim(0, 1)
    ax.set_ylabel("Fraction of Events")

    ax.grid(axis="y", alpha=0.4)
    ax.set_axisbelow(True)
    ax.set_box_aspect(1)
    ax.tick_params(axis="y", right=True)
    ax.bar_label(bar, fmt="{:.3f}")
    # ax.set_box_aspect(1)


def get_used_evts(dstr, etype, dtype, key):
    data_shape = np.load(BF_DATA / "indexed_1.00" / f"{dstr}_{etype}_{dtype}.npz")[key].shape
    dim = 1 if len(data_shape) == 1 else data_shape[0]
    data = [[] for _ in range(dim)]
    for invm in INVMS[:-1]:
        datum = np.load(BF_DATA / f"indexed_{invm:.2f}" / f"{dstr}_{etype}_{dtype}.npz")[key]
        datum = datum.reshape((-1, datum.shape[-1]))
        for ind, d in enumerate(datum):
            data[ind] += list(d)

    if dim == 1:
        return np.array(data)[0]
    return np.array(data)


def avail_used_data():
    return np.unique(["_".join(x.name.split("_")[:-2]) for x in (BF_DATA / "indexed_1.00").iterdir()])

In [ ]:
avail_used_data()

## For Brute Force of All Events

In [ ]:
etype, dtype, qcl = "ttbar", "parton", "QA_QA"
data = get_used_evts(dstr="min_bss", etype=etype, dtype=dtype, key=qcl)
matching_acc = sum(data == SYM_TRUE_BS_DICT[etype]) / len(data)
print(f"Efficiency: {100 * matching_acc:.2f}%")


# Functions to create histograms for the brute force methods
def bf_mass_hist(etype, dtype, qcl, lims=(0, 250)):
    data = get_used_evts(dstr="masses", etype=etype, dtype=dtype, key=f"min_{qcl}")
    # Mass distribution for min bitstrings
    fig, ax = plt.subplots(figsize=(10, 10))
    mass_histogram(ax, *data, etype=etype, lims=lims)

    return fig, ax, data


def bf_njet_hist(etype, dtype, qcl, bit="both"):
    # Number of jets histogram for min bitstrings
    if bit == "both":
        keys = [f"{qcl}0", f"{qcl}1"]
    else:
        keys = [f"{qcl}{bit}"]

    counts = np.zeros(NUM_FSP_DICT[etype] + 1)
    for key in keys:
        data = get_used_evts(dstr="num_jets", etype=etype, dtype=dtype, key=key)
        counts += np.sum(data.reshape(-1, 7), axis=0)

    fig, ax = plt.subplots(figsize=(10, 10))
    jet_histogram(ax, counts, max_jets=NUM_FSP_DICT[etype])

    return fig, ax


fig, ax, data = bf_mass_hist(etype=etype, dtype=dtype, qcl=qcl)
fig, ax = bf_njet_hist(etype=etype, dtype=dtype, qcl=qcl)

## For Hemisphere Method

In [ ]:
etype, dtype = "ttbar", "parton"

# Uses only events used in simulations
tot_cor, tot_evts = 0, 0
m1s, m2s = [], []
njets_0, njets_1 = [], []
for invm_path in BF_DATA.iterdir():
    if invm_path.name == "all_events":
        continue
    data = np.load(invm_path / f"hemisphere_{etype}_{dtype}.npz")
    bss, m1, m2 = data["bss"], data["m1s"], data["m2s"]
    m1s.append(m1)
    m2s.append(m2)
    njets_0.append(data["num_jets0"])
    njets_1.append(data["num_jets1"])
    tot_cor += sum(bss == swap(SYM_TRUE_BS_DICT[etype]))
    tot_evts += len(bss)

m1s = np.array(m1s).reshape(-1)
m2s = np.array(m2s).reshape(-1)
njet_data = {"num_jets0": np.sum(njets_0, axis=0), "num_jets1": np.sum(njets_1, axis=0)}
matching_acc = tot_cor / tot_evts

print(f"Efficiency: {100 * matching_acc:.2f}%")

fig, ax = plt.subplots(figsize=(10, 10))
# mass_histogram(ax, data["m1s"], data["m2s"], etype=etype, lims=(0, 500))
mass_histogram(ax, m1s, m2s, etype=etype, lims=(0, 500))

# Number of jets histogram for min bitstrings
fig, ax = plt.subplots(figsize=(10, 10))
bit = "both"
match bit:
    case "both":
        njet_data = np.sum(np.array(list(njet_data.values())), axis=0)
    case _:
        njet_data = njet_data[f"num_jets{bit}"]

jet_histogram(ax, njet_data, max_jets=NUM_FSP_DICT[etype])

## For algorithms

In [ ]:
alg = "falqon"
etype, dtype, qcl = "ttbar", "parton", "QA"
depth = 250
if alg == "falqon":
    acc_data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{qcl}.npz")["tot_correct_effs"][depth]
else:
    acc_data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{qcl}.npz")[f"tot_correct_effs_p{depth}"]
print(f"Efficiency: {100 * acc_data:.2f}%")


# Functions to create histograms for algorithm simulation data
def alg_mass_hist(etype, dtype, qcl, alg, depth, lims=(50, 500), bins=50):
    quadcoeff = qcl.split("_")[0]
    data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz")
    if alg == "falqon":
        m1s, m2s = (
            data["tot_m1s"][depth],
            data["tot_m2s"][depth],
        )
    else:
        m1s, m2s = (
            data[f"tot_m1s_p{depth}"],
            data[f"tot_m2s_p{depth}"],
        )
    fig, ax = plt.subplots(figsize=(10, 10))
    mass_histogram(ax, m1s, m2s, etype=etype, lims=lims)

    return fig, ax


def alg_njet_hist(etype, dtype, qcl, alg, depth, bit="both"):
    quadcoeff = qcl.split("_")[0]
    data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz")

    match alg.lower():
        # for FALQON
        case "falqon":
            if bit == "both":
                num_jets = data["tot_num_jets0"][depth] + data["tot_num_jets1"][depth]
            else:
                num_jets = data[f"tot_num_jets{bit}"][depth]
        # For other algs
        case _:
            if bit == "both":
                num_jets = data[f"tot_num_jets0_p{depth}"] + data[f"tot_num_jets1_p{depth}"]
            else:
                num_jets = data[f"tot_num_jets{bit}_p{depth}"]

    fig, ax = plt.subplots(figsize=(10, 10))
    jet_histogram(ax, num_jets, max_jets=NUM_FSP_DICT[etype])

    return fig, ax


fig, ax = alg_mass_hist(etype=etype, dtype=dtype, qcl=qcl, alg=alg, depth=depth, lims=(0, 250))
fig, ax = alg_njet_hist(etype=etype, dtype=dtype, qcl=qcl, alg=alg, depth=depth)

## Multiple algorithms on same jet histogram

In [ ]:
def multalg_njet_hist(algs, depths, etype, dtype, qcl, labels, hatches, colors, bit="both"):
    """
    Plots the jet histogram except for multiple algorithms at once.

    Parameters:
    algs - List of algorithm names to plot.
    depths - Equal length list of `algs` with the depth of each one.
    etype - Event type, e.g. "ttbar".
    dtype - Data dtype, e.g. "parton".
    qcl - Hamiltonian type + lambda value, e.g. "H0" or "QA_QA".
    labels - List of legend labels for each algorithm.
    hatches - List of hatches for each algorithm.
    colors - List of colors for each algorithm.
    bit (default "both") - Which particle to plot: bit=0 is particle 1, bit=1 is
        particle 2 and bit="both" is both particles.
    """
    # Any fractions under this threshold are removed to declutter the plot
    del_thresh = 0.004
    N_fsp = NUM_FSP_DICT[etype]
    width = 0.85 / len(labels)
    offset = -width * len(labels) / 2 + width / 2

    fig, ax = plt.subplots(figsize=(20, 10))

    # Data from algorithms
    num_jetss = []
    for ind, (alg, depth) in enumerate(zip(algs, depths)):
        data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{qcl}.npz")
        if alg == "falqon":
            num_jetss.append(data[f"tot_num_jets{bit}"][depth])
        else:
            num_jetss.append(data[f"tot_num_jets{bit}_p{depth}"])

    # Data from hemisphere/brute force
    hemisphere_num_jets, min_num_jets = [], []
    for invm in INVMS[:-1]:
        data = np.load(BF_DATA / f"indexed_{invm:.2f}" / f"hemisphere_{etype}_{dtype}.npz")
        hemisphere_num_jets.append(data[f"num_jets{bit}"])
        data = np.load(BF_DATA / f"indexed_{invm:.2f}" / f"num_jets_{etype}_{dtype}.npz")
        min_num_jets.append(data[f"{qcl}{bit}"])

    num_jetss.append(np.sum(hemisphere_num_jets, axis=0))
    num_jetss.append(np.sum(min_num_jets, axis=0))

    for ind, num_jets in enumerate(num_jetss):
        norm_num_jets = num_jets / sum(num_jets)
        # Remove bars that don't pass a threshold
        # So there aren't number labels like "0.00"
        below_thresh_vals = np.where(norm_num_jets < del_thresh)[0]
        norm_num_jets = np.delete(norm_num_jets, below_thresh_vals)
        jet_vals = np.delete(np.arange(N_fsp + 1), below_thresh_vals)

        bar = ax.bar(
            offset + jet_vals + (ind * width),
            norm_num_jets,
            width=width,
            hatch=hatches[ind],
            label=labels[ind],
            color=colors[ind],
        )
        ax.bar_label(bar, fmt="{:.2f}")

    ax.legend(fontsize=15, handletextpad=0.5, labelspacing=0.8, fancybox=True, shadow=True)

    # x-axis stuff
    ax.set_xlim(-0.5, N_fsp + 0.5)
    ax.set_xlabel("Number of jets", fontsize=13)
    # y-axis stuff
    ax.yaxis.set_major_locator(MultipleLocator(0.1))
    ax.set_ylim(0, 1)
    ax.set_ylabel("Fraction of Events", fontsize=13)

    ax.grid(axis="y", alpha=0.6)
    ax.set_axisbelow(True)
    ax.tick_params(axis="y", right=True)

    return fig, ax


fig, ax = multalg_njet_hist(
    algs=["qaoa", "maqaoa", "xqaoa", "falqon"],
    depths=[8, 8, 8, 300],
    etype="tW",
    dtype="parton",
    qcl="H1",
    bit="both",
    labels=[
        "QAOA",
        "ma-QAOA",
        "XQAOA",
        "FALQON",
        "Hemisphere method",
        "Hamiltonion Limit",
    ],
    hatches=["x", "++", "**", ".O", "*-", "x|"],
    colors=["#76B041", "#2E282A", "#E4572E", "#17BEBB", "#FFC914", "#93867F"],
)

# Testing Efficiency of Different Hamiltonians

## Used Functions

In [15]:
etype = "ttbar"
dtype = "parton"

In [16]:
# All possible permutation for the indices for one particle (is symmetric for the other)
zero_inds = []
one_inds = []
N = len(SYM_TRUE_BS_DICT[etype])
for bs in ["".join(x) for x in product(["0", "1"], repeat=N)][: 2 ** (N - 1)]:
    zero_inds.append([ind for ind, b in enumerate(bs) if b == "0"])
    one_inds.append([b for b in range(N) if b not in zero_inds[-1]])

zero_inds = zero_inds[1:-1]
one_inds = one_inds[1:-1]

In [17]:
# Hamiltonian parameterized by powers and coefficients
@njit
def param_hamil(p1s, p2s, alpha, lmbda, mpow, ppow):
    p1 = np.sum(p1s, axis=0)
    p2 = np.sum(p2s, axis=0)

    m1sq = np.dot(p1, METRIC * p1)
    m2sq = np.dot(p2, METRIC * p2)

    return alpha * (m1sq - m2sq) ** mpow + lmbda * (m1sq + m2sq) ** ppow

In [18]:
# Finds efficiency per bin by brute force for given Hamiltonian (parameters)
def find_eff(p4s, alpha, lmbdas, mpow, ppow, do_print=True):
    N_fsp = p4s.shape[1]
    if len(lmbdas) != len(p4s):
        print(f"\n{len(lmbdas)} | {len(p4s)}")
        assert False

    results = []
    min_Hs = []
    for ind, p4 in enumerate(p4s):
        lmbda = lmbdas[ind]
        if do_print:
            print(
                f"[α={alpha}, λ={lmbda:.4e}, powers=({mpow}, {ppow})] {ind + 1:,} / {len(p4s):,}",
                end="\r",
            )

        min_e, min_bs = float("inf"), ""
        for zero_ind, one_ind in zip(zero_inds, one_inds):
            zero_p4 = p4[zero_ind]
            one_p4 = p4[one_ind]

            res = param_hamil(zero_p4, one_p4, alpha, lmbda, mpow, ppow)
            if res < min_e:
                min_e = res
                min_bs = one_ind

        bs = ["0"] * N_fsp
        for b in min_bs:
            bs[b] = "1"
        results.append("".join(bs))
        min_Hs.append(min_e)
    if do_print:
        print()
    # Also return distribution of minimum energies
    return results, np.array(min_Hs)

## Calculate Data

In [19]:
correct_bs = SYM_TRUE_BS_DICT[etype]
p4s = np.load(EVT_DIR / f"{etype}_{dtype}.npy")
num_fsp, p4s, invms = format_p4s(p4s, return_extra=True)

# Kinematic values for constructing different lambdas
Ne = len(p4s)
min_Jij, max_Jij, avg_Jij = np.empty(Ne), np.empty(Ne), np.empty(Ne)
min_Pij, max_Pij, avg_Pij = np.empty(Ne), np.empty(Ne), np.empty(Ne)

Jijs, Pijs = get_Jijs_Pijs(p4s)
for ind, (Jij, Pij) in enumerate(zip(Jijs, Pijs)):
    min_Jij[ind] = np.min(Jij)
    max_Jij[ind] = np.max(Jij)
    avg_Jij[ind] = np.mean(Jij)
    min_Pij[ind] = np.min(Pij)
    max_Pij[ind] = np.max(Pij)
    avg_Pij[ind] = np.mean(Pij)

Below each section of two runs of `find_eff` uses a specific choice of $\lambda$ as described in the preceeding comment. The first `find_eff` uses the "QA" Hamiltonian
\begin{align*}
    H_{\rm QA}=(P_1^2-P_2^2)^2+\lambda(P_1^2+P_2^2)
\end{align*}
and the second uses the "$H_1$" Hamiltonian
\begin{align*}
    H_1=\lambda(P_1^2+P_2^2),
\end{align*}
i.e. the second term of $H_{\rm QA}$. Note that $\lambda$ must have dimension 2.

In [ ]:
# Don't use all for shorter run times
evts = Ne if Ne < 100_000 else 100_000
subset = RNG.choice(range(Ne), evts, replace=False)
sub_min_Jij, sub_max_Jij, sub_avg_Jij = (
    min_Jij[subset],
    max_Jij[subset],
    avg_Jij[subset],
)
sub_min_Pij, sub_max_Pij, sub_avg_Pij = (
    min_Pij[subset],
    max_Pij[subset],
    avg_Pij[subset],
)
sub_p4s = p4s[subset]
sub_invms = invms[subset]

# max == 1, min == 2, avg == 3
lambdas_jp11 = sub_max_Jij / sub_max_Pij
lambdas_jp12 = sub_max_Jij / sub_min_Pij
lambdas_jp13 = sub_max_Jij / sub_avg_Pij
lambdas_jp21 = sub_min_Jij / sub_max_Pij  # Definition from QA paper
lambdas_jp22 = sub_min_Jij / sub_min_Pij
lambdas_jp23 = sub_min_Jij / sub_avg_Pij
lambdas_jp31 = sub_avg_Jij / sub_max_Pij
lambdas_jp32 = sub_avg_Jij / sub_min_Pij
lambdas_jp33 = sub_avg_Jij / sub_avg_Pij
lambdas_p1 = sub_max_Pij
lambdas_p2 = sub_min_Pij
lambdas_p3 = sub_avg_Pij
p4s_choice = p4s[subset]

# Original Hamiltonian: (P1^2 - P2^2)^2
bs_0, H0 = find_eff(p4s_choice, alpha=1, lmbdas=[0] * len(p4s_choice), mpow=2, ppow=0)
bs_1, H1 = find_eff(p4s_choice, alpha=0, lmbdas=[1] * len(p4s_choice), mpow=0, ppow=2)

## Lambda choice -- λ=max(Jij)/max(Pij)
bs_m2p1_jp11, H_m2p1_jp11 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp11, mpow=2, ppow=1)
## Lambda choice -- λ=max(Jij)/min(Pij)
bs_m2p1_jp12, H_m2p1_jp12 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp12, mpow=2, ppow=1)
## Lambda choice -- λ=max(Jij)/avg(Pij)
bs_m2p1_jp13, H_m2p1_jp13 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp13, mpow=2, ppow=1)
## Lambda choice -- λ=min(Jij)/max(Pij)
bs_m2p1_jp21, H_m2p1_jp21 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp21, mpow=2, ppow=1)
## Lambda choice -- λ=min(Jij)/min(Pij)
bs_m2p1_jp22, H_m2p1_jp22 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp22, mpow=2, ppow=1)
## Lambda choice -- λ=min(Jij)/avg(Pij)
bs_m2p1_jp23, H_m2p1_jp23 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp23, mpow=2, ppow=1)
## Lambda choice --  λ=avg(Jij)/max(Pij)
bs_m2p1_jp31, H_m2p1_jp31 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp31, mpow=2, ppow=1)
## Lambda choice --  λ=avg(Jij)/min(Pij)
bs_m2p1_jp32, H_m2p1_jp32 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp32, mpow=2, ppow=1)
## Lambda choice --  λ=avg(Jij)/avg(Pij)
bs_m2p1_jp33, H_m2p1_jp33 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_jp33, mpow=2, ppow=1)
## Lambda choice -- λ=max(Pij)
bs_m2p1_p1, H_m2p1_p1 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_p1, mpow=2, ppow=1)
## Lambda choice -- λ=min(Pij)
bs_m2p1_p2, H_m2p1_p2 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_p2, mpow=2, ppow=1)
## Lambda choice -- λ=avg(Pij)
bs_m2p1_p3, H_m2p1_p3 = find_eff(p4s_choice, alpha=1, lmbdas=lambdas_p3, mpow=2, ppow=1)

## Plot Data

In [21]:
H0_str, H0_str_alt = "(P_1^2-P_2^2)^2", "H_0"
H1_str, H1_str_alt = "\lambda(P_1^2+P_2^2)", "H_1"
Hq_str, Hq_str_alt = f"{H0_str}+{H1_str}", r"H_{\rm tot}"
Jmin, Jmax, Javg = r"\min(J_{ij})", r"\max(J_{ij})", r"\overline{J_{ij}}"
Pmin, Pmax, Pavg = r"\min(P_{ij})", r"\max(P_{ij})", r"\overline{P_{ij}}"
l_colors = [
    "#002211",
    "#EF476F",
    "#06d6a0",
    "#118ab2",
    "#DCBF85",
    "#93A29B",
    "#B388EB",
    "#72DDF7",
    "#B97375",
    "#F9E900",
    "#F29559",
    "#FDC5F5",
    "#7F7B82",
]
H_hatches = ["-", "x", r"\\", "//"]
H_ls = ["solid", "dashed", "dotted"]
l_eq = lambda x, y=None: rf"\lambda={x}/{y}" if y is not None else rf"\lambda={x}"  # noqa: E731
H0_lw = 4
lw = 2

# [bit strings, label, alt label, lambda label, color, line style, alpha, line width, hatch, metadata label]
hamil_eff_data = [
    ## H0 Hamiltonian
    [bs_0, H0_str, H0_str_alt, "", l_colors[0], H_ls[0], 1, H0_lw, H_hatches[2], "H0"],
    ## H1 Hamiltonian
    [bs_1, f"({H1_str})^2", H1_str_alt, "", l_colors[0], H_ls[2], 1, H0_lw, H_hatches[3], "H1"],
    ## Lambda choice -- λ=max(Jij)/max(Pij)
    [bs_m2p1_jp11, Hq_str, Hq_str_alt, l_eq(Jmax, Pmax), l_colors[1], H_ls[0], 1, lw, H_hatches[1], "Hq_11"],
    ## Lambda choice -- λ=max(Jij)/min(Pij)
    [bs_m2p1_jp12, Hq_str, Hq_str_alt, l_eq(Jmax, Pmin), l_colors[2], H_ls[0], 1, lw, H_hatches[1], "Hq_12"],
    ## Lambda choice -- λ=max(Jij)/avg(Pij)
    [bs_m2p1_jp13, Hq_str, Hq_str_alt, l_eq(Jmax, Pavg), l_colors[3], H_ls[0], 1, lw, H_hatches[1], "Hq_13"],
    ## Lambda choice -- λ=min(Jij)/max(Pij)
    [bs_m2p1_jp21, Hq_str, Hq_str_alt, l_eq(Jmin, Pmax), l_colors[4], H_ls[0], 1, lw, H_hatches[1], "Hq_21"],
    ## Lambda choice -- λ=min(Jij)/min(Pij)
    [bs_m2p1_jp22, Hq_str, Hq_str_alt, l_eq(Jmin, Pmin), l_colors[5], H_ls[0], 1, lw, H_hatches[1], "Hq_22"],
    ## Lambda choice -- λ=min(Jij)/avg(Pij)
    [bs_m2p1_jp23, Hq_str, Hq_str_alt, l_eq(Jmin, Pavg), l_colors[6], H_ls[0], 1, lw, H_hatches[1], "Hq_23"],
    ## Lambda choice --  λ=avg(Jij)/max(Pij)
    [bs_m2p1_jp31, Hq_str, Hq_str_alt, l_eq(Javg, Pmax), l_colors[7], H_ls[0], 1, lw, H_hatches[1], "Hq_31"],
    ## Lambda choice --  λ=avg(Jij)/min(Pij)
    [bs_m2p1_jp32, Hq_str, Hq_str_alt, l_eq(Javg, Pmin), l_colors[8], H_ls[0], 1, lw, H_hatches[1], "Hq_32"],
    ## Lambda choice --  λ=avg(Jij)/avg(Pij)
    [bs_m2p1_jp33, Hq_str, Hq_str_alt, l_eq(Javg, Pavg), l_colors[9], H_ls[0], 1, lw, H_hatches[1], "Hq_33"],
    ## Lambda choice -- λ=max(Pij)
    [bs_m2p1_p1, Hq_str, Hq_str_alt, l_eq(Pmax), l_colors[10], H_ls[0], 1, lw, H_hatches[1], "Hq_1"],
    ## Lambda choice -- λ=min(Pij)
    [bs_m2p1_p2, Hq_str, Hq_str_alt, l_eq(Pmin), l_colors[11], H_ls[0], 1, lw, H_hatches[1], "Hq_2"],
    ## Lambda choice -- λ=avg(Pij)
    [bs_m2p1_p3, Hq_str, Hq_str_alt, l_eq(Pavg), l_colors[12], H_ls[0], 1, lw, H_hatches[1], "Hq_3"],
]

In [22]:
# Find efficiencies
percentages, counts, tots = [], [], []
correct_bs = SYM_TRUE_BS_DICT[etype]
for hamil_eff_datum in hamil_eff_data:
    bit_strs = hamil_eff_datum[0]
    count = [0] * len(INVMS)
    tot = [0] * len(INVMS)
    for ind, (bs, invm) in enumerate(zip(bit_strs, sub_invms)):
        invm_bin = np.searchsorted(INVMS, invm) - 1

        tot[invm_bin] += 1
        if bs == correct_bs:
            count[invm_bin] += 1

    percentages.append([c / t for c, t in zip(count, tot)])
    counts.append(count)
    tots.append(tot)

percentages = np.array(percentages)
counts = np.array(counts)
tots = np.array(tots)

### Efficiency Plot

In [ ]:
INVMS_EXT = np.concatenate((INVMS, [3.5]))
fig, ax = plt.subplots(figsize=(15, 12))

handles = []
# sorted by max average efficiencies
sorted_inds = np.arange(len(percentages))  # np.argsort(np.mean(percentages, axis=1))[::-1]
cutoff = 100
# ignores = [] #["Hq_2", "Hq_33", "Hq_32"]
includes = ["H0", "H1", "Hq_21", "Hq_2", "Hq_3"]
ind = 0
for sorted_ind in sorted_inds:
    if ind == cutoff:
        break

    percs = percentages[sorted_ind]
    eff_datum = hamil_eff_data[sorted_ind]
    # if eff_datum[-1] in ignores:
    if eff_datum[-1] not in includes:
        continue

    label = f"${eff_datum[2]}$" + (f", ${eff_datum[3]}$" if eff_datum[3] else "")
    color = eff_datum[4]
    ls = eff_datum[5]
    alpha = eff_datum[6]
    lw = eff_datum[7]

    histbar(
        ax,
        xs=INVMS_EXT,
        ys=percs,
        capends=False,
        label_type="explicit",
        color=color,
        ls=ls,
        lw=lw,
        alpha=alpha,
        dash_capstyle="round",
    )
    handles.append(Line2D([0], [0], color=color, ls=ls, label=label))
    ind += 1

ax.set_xlim(INVMS_EXT[0], INVMS_EXT[-1])
ax.set_ylim(0, 1)
ax.set_xlabel("$m_{AB}/(m_A+m_B)$", fontsize=15)
ax.set_ylabel("Efficiency", fontsize=15)
ax.tick_params(top=True, right=True, which="both")
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_major_locator(MultipleLocator(0.25))
# Set custom label for last tick
xlabels = [item.get_text() for item in ax.get_xticklabels()]
xlabels[-1] = "3.0+ →"
ax.set_xticklabels(xlabels)

ax.tick_params(axis="both", labelsize=11)
ax.grid(which="major", alpha=0.5)
ax.set_axisbelow(True)

leg = fig.legend(
    handles=handles,
    loc="center left",
    bbox_to_anchor=(0.92, 0.5),
    shadow=True,
    facecolor="#eeeeee",
    handlelength=5,
    fancybox=True,
    fontsize=12,
    ncols=1,
)
for line in leg.get_lines():
    line.set_linewidth(4)

# plt.savefig(PLOTS_DIR / "misc" / f"eff_top5_{etype}_{dtype}.png", bbox_inches="tight")

### Bar Plot

In [ ]:
labels, colors, hatches, bar_counts, bar_tots = [], [], [], [], []
ignores = []  # ["Hq_2", "Hq_33", "Hq_32"]
for eff_datum, count, tot in zip(hamil_eff_data, counts, tots):
    if (eff_datum[-1] in ignores) or (sum(count) / sum(tot) < 0.1):
        continue
    labels.append(f"${eff_datum[2]}$" + (f"\n${eff_datum[3]}$" if eff_datum[3] else ""))
    colors.append(eff_datum[4])
    hatches.append(eff_datum[8])
    bar_counts.append(count)
    bar_tots.append(tot)

tot_percs = [sum(count) / sum(tot) for count, tot in zip(bar_counts, bar_tots)]

fig, ax = plt.subplots(figsize=(16, 12))

ax.barh(labels, tot_percs, color=colors, hatch=hatches)
ax.set_xlim(0, 1)
ax.set_title("Total efficiency")
ax.xaxis.set_major_locator(MultipleLocator(0.1))
ax.xaxis.set_minor_locator(MultipleLocator(0.05))
ax.grid(axis="x", alpha=0.7)
ax.grid(axis="x", which="minor", alpha=0.5)
ax.set_axisbelow(True)
ax.tick_params(top=True, labeltop=True, which="both")

# plt.savefig(PLOTS_DIR / "misc" / f"tot_eff_{etype}_{dtype}.png", bbox_inches="tight")

## Scan over $\lambda_2$ for efficiency

In [26]:
etype = "ttbar"
num_lmbdas = 30
min_inv_lmbda, max_inv_lmbda = 10**-2, 10**6
inv_lmbda2s = np.geomspace(min_inv_lmbda, max_inv_lmbda, num_lmbdas)
correct_bs = SYM_TRUE_BS_DICT[etype]

### For QA Paper $\lambda$

In [ ]:
bit_strs = []
lambdas = lambdas_jp21
for ind, inv_lmbda2 in enumerate(inv_lmbda2s):
    print(
        f"[{ind + 1:>{len(str(num_lmbdas))}}/{num_lmbdas}] -- 1/λ = {inv_lmbda2:.3e}",
        end="\r",
    )
    bit_str, _ = find_eff(sub_p4s, alpha=1, lmbdas=lambdas / inv_lmbda2, mpow=2, ppow=1, do_print=False)
    bit_strs.append(bit_str)

bit_strs = np.array(bit_strs)

In [28]:
# Find efficiencies
percentages, tot_percentages = [], []
# Total efficiency (ignoring inv mass bins)
tot_eff = [0] * len(bit_strs)
for dind, data in enumerate(bit_strs):
    count = [0] * len(INVMS)
    tot = [0] * len(INVMS)
    for ind, bs in enumerate(data):
        invm = sub_invms[ind]
        invm_bin = np.searchsorted(INVMS, invm) - 1

        tot[invm_bin] += 1
        if bs == correct_bs:
            count[invm_bin] += 1
            tot_eff[dind] += 1

    percentages.append([c / t for c, t in zip(count, tot)])
    tot_percentages.append(sum(count) / sum(tot))

percentages = np.array(percentages)
tot_percentages = np.array(tot_percentages)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

lambda_label = "\lambda=\min(J_{ij})/\max(P_{ij})"
colors = multifader(colors=["#5D2E46", "#E53D00"], fractions=np.linspace(0, 1, len(INVMS) - 1))
for effs, color, invm_lo, invm_hi in zip(percentages.T[:-1], colors, INVMS[:-1], INVMS[1:]):
    ax.scatter(inv_lmbda2s, effs, c=color)
    ax.plot(
        inv_lmbda2s,
        effs,
        c=color,
        label=f"${invm_lo:.2f}<m_{{tt}}/2m_t<{invm_hi:.2f}$",
    )
ax.plot(
    inv_lmbda2s,
    tot_percentages,
    c="k",
    alpha=0.7,
    ls=(5, (10, 3)),
    lw=4,
    label="Total",
    dash_capstyle="round",
)

ax.set_ylabel("Efficiency", fontsize=15)
ax.set_ylim(0, 1.02)
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xscale("log")
ax.xaxis.set_major_locator(LogLocator(numticks=10))
ax.xaxis.set_minor_locator(LogLocator(numticks=1000, subs="auto"))
ax.set_xlim(0.95 * min_inv_lmbda, 1.05 * max_inv_lmbda)
ax.set_xlabel("$1/\lambda_2$", fontsize=15)
ax.set_title(f"{dtype.capitalize()} events, ${lambda_label}$", fontsize=18)
ax.grid(alpha=0.6)
ax.grid(which="minor", alpha=0.6, ls="dotted")
ax.legend()

# plt.savefig(PLOTS_DIR / "misc" / "lambda2_tot_eff_minJij_by_maxPij_ttbar_parton.png", bbox_inches="tight")

### For Best Performing $\lambda$

In [ ]:
bit_strs2 = []
lambdas = lambdas_p3
for ind, inv_lmbda2 in enumerate(inv_lmbda2s):
    print(
        f"[{ind + 1:>{len(str(num_lmbdas))}}/{num_lmbdas}] -- 1/λ = {inv_lmbda2:.3e}",
        end="\r",
    )
    bit_str, _ = find_eff(sub_p4s, alpha=1, lmbdas=lambdas / inv_lmbda2, mpow=2, ppow=1, do_print=False)
    bit_strs2.append(bit_str)

bit_strs2 = np.array(bit_strs2)

In [31]:
# Find efficiencies
percentages2, tot_percentages2 = [], []
# Total efficiency (ignoring inv mass bins)
for dind, data in enumerate(bit_strs2):
    count = [0] * len(INVMS)
    tot = [0] * len(INVMS)
    for ind, bs in enumerate(data):
        invm = sub_invms[ind]
        invm_bin = np.searchsorted(INVMS, invm) - 1

        tot[invm_bin] += 1
        if bs == correct_bs:
            count[invm_bin] += 1

    percentages2.append([c / t for c, t in zip(count, tot)])
    tot_percentages2.append(sum(count) / sum(tot))

percentages2 = np.array(percentages2)
tot_percentages2 = np.array(tot_percentages2)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))

lambda_label = "\lambda=\overline{P_{ij}}"
colors = multifader(colors=["#4C6663", "#3772FF"], fractions=np.linspace(0, 1, len(INVMS) - 1))
for effs, color, invm_lo, invm_hi in zip(percentages2.T[:-1], colors, INVMS[:-1], INVMS[1:]):
    ax.scatter(inv_lmbda2s, effs, c=color)
    ax.plot(
        inv_lmbda2s,
        effs,
        c=color,
        label=f"${invm_lo:.2f}<m_{{AB}}/(m_A+m_B)<{invm_hi:.2f}$",
    )
ax.plot(
    inv_lmbda2s,
    tot_percentages2,
    c="k",
    alpha=0.7,
    ls=(5, (10, 3)),
    lw=4,
    label="Total",
    dash_capstyle="round",
)

ax.set_ylabel("Efficiency", fontsize=15)
ax.set_ylim(0, 1.02)
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xscale("log")
ax.xaxis.set_major_locator(LogLocator(numticks=10))
ax.xaxis.set_minor_locator(LogLocator(numticks=1000, subs="auto"))
ax.set_xlim(0.95 * min_inv_lmbda, 1.05 * max_inv_lmbda)
ax.set_xlabel("$1/\lambda_2$", fontsize=15)
ax.set_title(f"{dtype.capitalize()} events, ${lambda_label}$", fontsize=18)
ax.grid(alpha=0.6)
ax.grid(which="minor", alpha=0.6, ls="dotted")
ax.legend()

# plt.savefig(PLOTS_DIR / "misc" / "lambda2_tot_eff_avgPij_ttbar_parton.png", bbox_inches="tight")

# Success Rate (Efficiency) vs Depth

In [ ]:
def eff_vs_numparams(etype, dtype, qcl, algs, colors, invm_val):
    x_off = 0.5
    quadcoeff = qcl.split("_")[0]

    alg_depths = []
    alg_min = []
    alg_correct = []
    min_depth, max_depth = 10000, 0
    for alg in algs:
        # File with all data for specific params
        data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz")
        # Get the depths used for specific algorithm
        depths = [int(key.split("_p")[-1]) for key in data.keys() if key.startswith("m1s")]
        alg_depths.append(depths)
        # Find min and max for setting limits
        min_depth = min(min_depth, min(depths))
        max_depth = max(max_depth, max(depths))

        min_effs, correct_effs = [], []
        for depth in depths:
            # Collect fo all
            if invm_val == "all":
                min_effs.append(data[f"tot_min_effs_p{depth}"])
                if etype != "6jet":
                    correct_effs.append(data[f"tot_correct_effs_p{depth}"])
            # Collect for specific invariant mass range
            else:
                min_effs.append(data[f"min_effs_p{depth}"][INVMS.index(invm_val)])
                if etype != "6jet":
                    correct_effs.append(data[f"correct_effs_p{depth}"][INVMS.index(invm_val)])

        alg_min.append(min_effs)
        alg_correct.append(correct_effs)

    # Make plot
    fig, ax = plt.subplots(figsize=(12, 8))
    # Get data for efficiency for brute force
    if etype != "6jet":
        tot_count, tot_N_evts = 0, 0
        if invm_val == "all":
            for invm in INVMS[:-1]:
                data = np.load(BF_DATA / f"indexed_{invm:.2f}" / f"effs_{etype}_{dtype}.npz")
                tot_count += data[qcl][1]
                tot_N_evts += data[qcl][2]
        else:
            data = np.load(BF_DATA / f"indexed_{invm_val:.2f}" / f"effs_{etype}_{dtype}.npz")
            tot_count += data[qcl][1]
            tot_N_evts += data[qcl][2]
        # Create greyed out area of Hamiltonian limit at top
        ax.axhline(tot_count / tot_N_evts, c="k")
        ax.fill_between(
            x=[min_depth - x_off, max_depth + x_off],
            y1=tot_count / tot_N_evts,
            y2=1,
            color="k",
            alpha=0.4,
            facecolor="#bbbbbb",
            hatch="//",
        )

    # Plot the data
    for ind, alg in enumerate(algs):
        depths = alg_depths[ind]
        if etype != "6jet":
            data_correct = alg_correct[ind]
            ax.plot(
                depths,
                data_correct,
                marker="o",
                c=colors[ind],
                ls="dotted",
                label=f"{alg.upper()}",
            )
        data_min = alg_min[ind]
        ax.plot(depths, data_min, marker="o", c=colors[ind], ls="solid")

    # Axis stuff
    ax.set_xlabel("Depth $p$")
    ax.set_xlim(min_depth - x_off, max_depth + x_off)
    ax.xaxis.set_major_locator(FixedLocator(np.unique(alg_depths[0])))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.set_ylabel("Success Rate")
    ax.set_ylim(0, 1)
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax.grid(axis="y", alpha=0.7)
    ax.grid(which="minor", axis="y", alpha=0.7, ls=":")

    # Legend stuff
    alg_handles = [Patch(color=color, label=alg.upper()) for color, alg in zip(colors, algs)]
    dtype_handles = [
        Line2D([0], [0], ls="solid", label="Minimum", color="k"),
        Line2D([0], [0], ls="dotted", label="Correct", color="k"),
    ]
    dtype_leg = ax.legend(
        handles=dtype_handles,
        bbox_to_anchor=(0.815, 0),
        loc="lower right",
        title="Efficiency Type",
    )
    ax.add_artist(dtype_leg)

    ax.legend(
        handles=alg_handles,
        loc="lower right",
        title="Algorithm",
    )

    # Title
    unit = {
        "ttbar": "m_{tt}/(2m_t)",
        "tW": "m_{tW}/(m_t+m_W)",
        "6jet": "m_{tt}/(2m_t)",
    }[etype]
    if invm_val == "all":
        invm_range = "1.00-3.00"
    else:
        invm_range = f"{invm_val:.2f}-{INVMS[INVMS.index(invm_val) + 1]:.2f}"
    ax.set_title(f"Invariant Mass Range: ${invm_range}$ $[{unit}]$", pad=15)

    return fig, ax


fig, ax = eff_vs_numparams(
    etype="ttbar",
    dtype="parton",
    qcl="QA_QA",
    algs=["qaoa", "maqaoa", "xqaoa"],
    colors=["#3F88C5", "#AB2346", "#5F00BA"],
    invm_val="all",
)

# $\Delta E$ versus Depth

In [ ]:
def deltaE_vs_numparams(etype, dtype, qcl, algs, colors, invm_val):
    quadcoeff = qcl.split("_")[0]
    # Will show 2nd axis for fraction where deltaE=0 (equivalent to success rate)
    show_frac0 = False
    x_off = 0.5

    # Plot it
    fig, ax = plt.subplots(figsize=(12, 8))
    if show_frac0:
        ax2 = ax.twinx()

    alg_depths = []
    min_depth, max_depth = 100000, 0
    # Collect data and plot it in the same for loop
    for ind, alg in enumerate(algs):
        data = np.load(ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz")
        # Get data for depths
        depths = [int(key.split("_p")[-1]) for key in data.keys() if key.startswith("m1s")]
        min_depth = min(min_depth, min(depths))
        max_depth = max(max_depth, max(depths))
        alg_depths.append(depths)

        correct_deltaEs, min_deltaEs = [], []
        correct_fracs0, min_fracs0 = [], []
        # Collect data for each depth
        for depth in depths:
            if invm_val == "all":
                min_deltaE = data[f"tot_min_deltaE_p{depth}"]
                if etype != "6jet":
                    correct_deltaE = data[f"tot_correct_deltaE_p{depth}"]
            else:
                min_deltaE = data[f"min_deltaE_p{depth}"][INVMS.index(invm_val)]
                if etype != "6jet":
                    correct_deltaE = data[f"correct_deltaE_p{depth}"][INVMS.index(invm_val)]
            min_deltaEs.append(min_deltaE)
            min_fracs0.append((len(min_deltaE) - np.count_nonzero(min_deltaE)) / len(min_deltaE))
            if etype != "6jet":
                correct_deltaEs.append(correct_deltaE)
                correct_fracs0.append((len(correct_deltaE) - np.count_nonzero(correct_deltaE)) / len(correct_deltaE))

        # Looking at average deltaE
        avg_min_deltaEs = np.mean(min_deltaEs, axis=1)
        ax.plot(depths, avg_min_deltaEs / 1e9, c=colors[ind], ls="dotted", marker="o")
        if etype != "6jet":
            avg_correct_deltaEs = np.mean(correct_deltaEs, axis=1)
            ax.plot(depths, avg_correct_deltaEs / 1e9, c=colors[ind], ls="solid", marker="o")
        if show_frac0:
            ax2.plot(depths, min_fracs0, c=colors[ind], ls="dashed")
            if etype != "6jet":
                ax2.plot(depths, correct_fracs0, c=colors[ind], ls="dashed")

    # Label stuff
    ax.set_xlabel("Depth $p$")
    ax.set_xlim(min_depth - x_off, max_depth + x_off)
    ax.xaxis.set_major_locator(FixedLocator(np.unique(alg_depths)))
    ax.xaxis.set_minor_locator(MultipleLocator(1))
    ax.set_ylabel("Average $\Delta E$ (GeV)")
    ax.set_yscale("log")
    ax.grid(axis="y", alpha=0.6)
    ax.grid(which="minor", axis="y", alpha=0.6, ls="dotted")
    ax.tick_params(top=True, labeltop=True)
    if show_frac0:
        ax2.set_ylim(0, 1)
        ax2.set_ylabel("Fraction of $\Delta E=0$")
        ax2.yaxis.set_minor_locator(MultipleLocator(0.1))

    # Legend stuff
    alg_handles = [Patch(color=color, label=alg.upper()) for color, alg in zip(colors, algs)]
    dtype_handles = [
        Line2D([0], [0], ls="solid", label="Correct", color="k"),
        Line2D([0], [0], ls="dotted", label="Minimum", color="k"),
    ]
    dtype_leg = ax.legend(
        handles=dtype_handles,
        bbox_to_anchor=(0.83, 1),
        loc="upper right",
        title="Efficiency Type",
    )
    ax.add_artist(dtype_leg)

    ax.legend(
        handles=alg_handles,
        loc="upper right",
        title="Algorithm",
    )

    # Title
    unit = {
        "ttbar": "m_{tt}/(2m_t)",
        "tW": "m_{tW}/(m_t+m_W)",
        "6jet": "m_{tt}/(2m_t)",
    }[etype]
    if invm_val == "all":
        invm_range = "1.00-3.00"
    else:
        invm_range = f"{invm_val:.2f}-{INVMS[INVMS.index(invm_val) + 1]:.2f}"
    ax.set_title(f"Invariant Mass Range: ${invm_range}$ $[{unit}]$")

    return fig, ax


fig, ax = deltaE_vs_numparams(
    etype="ttbar",
    dtype="parton",
    qcl="H0",
    algs=["qaoa", "maqaoa", "xqaoa"],
    colors=["#3F88C5", "#AB2346", "#5F00BA"],
    invm_val="all",
)


# FALQON Success Rate vs Depth

In [ ]:
etype, dtype, quadcoeff = "ttbar", "parton", "H0"
data = np.load(ALG_DATA / f"falqon_{etype}_{dtype}_{quadcoeff}.npz")
colors = ["#264653", "#2a9d8f", "#e9c46a", "#e76f51", "#780000", "#99582a"]

fig, ax = plt.subplots(figsize=(18, 10))

max_depth = 0
test_invm = []
# Plot efficiency lines
for ind, color in enumerate(colors):
    correct_ys = data["correct_effs"][:, ind]
    min_ys = data["min_effs"][:, ind]
    max_depth = max(max_depth, len(correct_ys), len(min_ys))
    ax.plot(range(1, len(correct_ys) + 1), correct_ys, c=color, lw=2, ls="solid")
    ax.plot(range(1, len(min_ys) + 1), min_ys, c=color, lw=2, ls="dotted")
    test_invm.append(min_ys)

correct_ys = data["tot_correct_effs"]
min_ys = data["tot_min_effs"]
ax.plot(range(1, len(correct_ys) + 1), correct_ys, c="k", lw=3, ls="solid")
ax.plot(range(1, len(min_ys) + 1), min_ys, c="k", lw=3, ls="dotted")

tot_count, tot_N_evts = 0, 0
for invm, color in zip(INVMS[:-1], colors):
    data = np.load(BF_DATA / f"indexed_{invm:.2f}" / f"effs_{etype}_{dtype}.npz")
    tot_count += data[quadcoeff][1]
    tot_N_evts += data[quadcoeff][2]

    # Create greyed out area of Hamiltonian limit at top
    # ax.axhline(tot_count / tot_N_evts, color=color, lw=4)
    # ax.fill_between(
    #     x=[-max_depth / 200, max_depth],
    #     y1=data[quadcoeff][1] / data[quadcoeff][2],
    #     y2=1,
    #     color=color,
    #     alpha=1,
    #     facecolor="#dddddd",
    #     hatch="//"
    # )
ax.axhline(tot_count / tot_N_evts, lw=3, color="k")
# ax.fill_between(
#     x=[-max_depth / 200, max_depth],
#     y1=tot_count / tot_N_evts,
#     y2=1,
#     color="k",
#     alpha=1,
#     facecolor="#dddddd",
#     hatch="//",
# )

ax.set_ylim(0.00, 1)
# ax.set_yscale("log")
ax.set_ylabel("Success rate")
ax.yaxis.set_major_locator(MultipleLocator(0.1))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_xlim(-max_depth / 200, 600)  # max_depth)
# ax.set_xscale("log")
ax.set_xlabel("Depth $p$")
ax.xaxis.set_major_locator(MultipleLocator(200))
ax.xaxis.set_minor_locator(MultipleLocator(100))

ax.grid(alpha=0.8)
ax.grid(which="minor", axis="x", alpha=0.8, ls=":")
ax.grid(which="minor", axis="y", alpha=0.4)

# Legend stuff
unit = {"ttbar": "m_{tt}/(2m_t)", "tW": "m_{tW}/(m_t+m_W)"}[etype]
invm_handles = [Patch(color=color, label=f"${invm_lo:.2f}<{unit}<{invm_hi:.2f}$") for color, invm_lo, invm_hi in zip(colors, INVMS[:-1], INVMS[1:])]
invm_handles.append(Patch(color="k", label=f"$1.00<{unit}<3.00$"))
dtype_handles = [
    Line2D([0], [0], ls="solid", label="Correct", color="k"),
    Line2D([0], [0], ls="dotted", label="Minimum", color="k"),
]
dtype_leg = ax.legend(
    handles=dtype_handles,
    loc="upper right",
    title="Efficiency Type",
)
ax.add_artist(dtype_leg)

ax.legend(
    handles=invm_handles,
    loc="lower right",
    title="Invariant Mass Ranges",
)

# FALQON $\Delta E$ vs Depth

In [ ]:
etype, dtype, quadcoeff = "ttbar", "parton", "H0"
data = np.load(ALG_DATA / f"falqon_{etype}_{dtype}_{quadcoeff}.npz")
colors = ["#264653", "#2a9d8f", "#e9c46a", "#e76f51", "#780000", "#99582a"]

fig, ax = plt.subplots(figsize=(18, 10))

max_depth = 0
for ind, color in enumerate(colors):
    correct_deltaEs = np.mean(data["correct_deltaE"][:, ind], axis=1)
    min_deltaEs = np.mean(data["min_deltaE"][:, ind], axis=1)

    max_depth = max(max_depth, len(correct_deltaEs), len(min_deltaEs))
    ax.plot(
        range(1, len(correct_deltaEs) + 1),
        correct_deltaEs / 1e9,
        c=color,
        lw=2,
        ls="dotted",
    )
    ax.plot(
        range(1, len(min_deltaEs) + 1),
        min_deltaEs / 1e9,
        c=color,
        lw=2,
        ls="solid",
        label=invm,
    )

tot_correct_deltaEs = np.mean(data["tot_correct_deltaE"], axis=1)
tot_min_deltaEs = np.mean(data["tot_min_deltaE"], axis=1)
ax.plot(
    range(1, len(tot_correct_deltaEs) + 1),
    tot_correct_deltaEs / 1e9,
    c="k",
    lw=3,
    ls="dotted",
)
ax.plot(range(1, len(tot_min_deltaEs) + 1), tot_min_deltaEs / 1e9, c="k", lw=3, ls="solid")

# Label stuff
ax.set_xlabel("Depth $p$")
ax.set_xlim(-max_depth / 200, max_depth)
ax.set_ylabel("Average $\Delta E$ (GeV)")
ax.set_yscale("log")
ax.grid(axis="y", alpha=0.6)
ax.grid(which="minor", axis="y", alpha=0.6, ls="dotted")
ax.tick_params(top=True, labeltop=True)

unit = {"ttbar": "m_{tt}/(2m_t)", "tW": "m_{tW}/(m_t+m_W)"}[etype]
alg_handles = [Patch(color=color, label=f"${invm_lo:.2f}<{unit}<{invm_hi:.2f}$") for color, invm_lo, invm_hi in zip(colors, INVMS[:-1], INVMS[1:])]
dtype_handles = [
    Line2D([0], [0], ls="solid", label="Correct", color="k"),
    Line2D([0], [0], ls="dotted", label="Minimum", color="k"),
]
dtype_leg = ax.legend(
    handles=dtype_handles,
    bbox_to_anchor=(0.80, 1),
    loc="upper right",
    title="Efficiency Type",
)
ax.add_artist(dtype_leg)

ax.legend(
    handles=alg_handles,
    loc="upper right",
    title="Invariant Mass Ranges",
)

# (Average) Energy vs Bit Strings

In [37]:
def eng_vs_bs(etype, dtype, qcl, percents=(10, 20, 30), add_extrs=False):
    data = np.load(BF_DATA / "all_events" / f"bs_engs_{etype}_{dtype}.npz")

    bss = data["bss"]
    # Energies per bit string rather than per event
    bs_energies = data[qcl].T
    mean_bs_engs = np.mean(bs_energies, axis=1)
    median_bs_engs = np.median(bs_energies, axis=1)
    # 1% from the inwards to the sorted array from the max/min
    perc_mins_bs_engs = [np.sort(bs_energies, axis=1)[:, int(bs_energies.shape[1] / (100 / percent))] for percent in percents]
    perc_maxs_bs_engs = [np.sort(bs_energies, axis=1)[:, -int(bs_energies.shape[1] / (100 / percent))] for percent in percents]

    sorted_inds = np.argsort(mean_bs_engs)
    sorted_bss = bss[sorted_inds]
    sorted_mean = mean_bs_engs[sorted_inds]
    sorted_median = median_bs_engs[sorted_inds]
    sorted_mins = [perc_min_bs_engs[sorted_inds] for perc_min_bs_engs in perc_mins_bs_engs]
    sorted_maxs = [perc_max_bs_engs[sorted_inds] for perc_max_bs_engs in perc_maxs_bs_engs]

    mean_col = "#353238"
    median_col = "#A75512"
    boundary_col = "#BE5A38"
    fill_col = "#BE7C4D"

    fig, ax = plt.subplots(figsize=(26, 12))
    xs = list(range(0, 2 ** NUM_FSP_DICT[etype]))
    handles = []

    ax.plot(sorted_mean, c=mean_col, lw=3, marker="d")
    handles.append(Line2D([0], [0], label="Mean", lw=3, c=mean_col, marker="d"))
    ax.plot(sorted_median, c=median_col, lw=2, marker="d")
    handles.append(Line2D([0], [0], label="Median", lw=2, c=median_col, marker="d"))
    if add_extrs:
        min_bs_engs = np.min(bs_energies, axis=1)
        max_bs_engs = np.max(bs_energies, axis=1)
        sorted_min = min_bs_engs[sorted_inds]
        sorted_max = max_bs_engs[sorted_inds]
        ax.plot(sorted_min, c=boundary_col, lw=4)
        ax.plot(sorted_max, c=boundary_col, lw=4)
    for ind, percent in enumerate(percents):
        alpha = 0.3 * (ind + 1) - 0.1
        ax.fill_between(x=xs, y1=sorted_mins[ind], y2=sorted_maxs[ind], color=fill_col, alpha=alpha)
        handles.append(Patch(label=f"{100 - 2 * percent}% within", color=fill_col, alpha=alpha))

    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.set_xlim(0, 2 ** NUM_FSP_DICT[etype] - 1)
    ax.set_xticks(xs)
    ax.set_xticklabels(sorted_bss, rotation=60, ha="right", fontsize=12)
    ax.set_xlabel("Bit string", labelpad=10)

    ax.set_yscale("log")
    ax.set_ylabel("Energy (eV)")
    # ax.set_ylim(1e4, 1e6)

    ax.grid(axis="y", which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", ls="dashed", alpha=0.6)
    ax.grid(axis="x", alpha=0.4)
    ax.set_axisbelow(True)

    ax.legend(handles=handles, loc="lower right")

    return fig, ax

In [ ]:
etype = "ttbar"
dtype = "parton"
qcl = "QA_QA"
fig, ax = eng_vs_bs(etype=etype, dtype=dtype, qcl=qcl)

In [ ]:
etypes = EVENT_CHOICES
dtype = "parton"
colors = ["#002642", "#840032", "#e59500"]
labels = [r"$t\bar{t}$", "$tW$", "$6$ jet"]
markers = ["o", "D", "*"]
markersizes = [7, 7, 15]
qcl = "QA_QA"
num_states = 32


def multeng_vs_bs(etypes, dtype, qcl, num_states, labels, colors, markers, markersizes):
    label_dict = {"ttbar": r"$t\bar{t}$", "tW": "$tW$", "6jet": "$6$ jet"}
    fig, ax = plt.subplots(figsize=(26, 12))
    handles = []
    for etype, label, color, marker, ms in zip(etypes, labels, colors, markers, markersizes):
        label = label_dict[etype]
        data = np.load(BF_DATA / "all_events" / f"bs_engs_{etype}_{dtype}.npz")

        # Energies per bit string rather than per event
        bs_energies = data[qcl].T
        mean_bs_engs = np.mean(bs_energies, axis=1)
        median_bs_engs = np.median(bs_energies, axis=1)
        # 1% from the inwards to the sorted array from the max/min
        percent = 25
        perc_min_bs_engs = np.sort(bs_energies, axis=1)[:, int(bs_energies.shape[1] / (100 / percent))]
        perc_max_bs_engs = np.sort(bs_energies, axis=1)[:, -int(bs_energies.shape[1] / (100 / percent))]

        sorted_inds = np.argsort(mean_bs_engs)[:num_states]
        sorted_mean = mean_bs_engs[sorted_inds]
        sorted_median = median_bs_engs[sorted_inds]
        sorted_min = perc_min_bs_engs[sorted_inds]
        sorted_max = perc_max_bs_engs[sorted_inds]

        ax.plot(sorted_mean, c=color, lw=3, marker=marker, markersize=ms)
        handles.append(
            Line2D(
                [0],
                [0],
                label=f"{label} Mean",
                lw=3,
                c=color,
                marker=marker,
                markersize=ms,
            )
        )
        ax.plot(sorted_median, c=color, lw=2, ls="dashed", marker=marker, markersize=ms)
        handles.append(
            Line2D(
                [0],
                [0],
                label=f"{label} Median",
                lw=2,
                ls="dashed",
                c=color,
                marker=marker,
                markersize=ms,
            )
        )
        xs = list(range(0, num_states))
        ax.fill_between(x=xs, y1=sorted_min, y2=sorted_max, color=color, alpha=0.3)
        ax.plot(sorted_min, c=color, alpha=0.5)
        ax.plot(sorted_max, c=color, alpha=0.5)

    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.set_xlim(0 - 0.1, num_states - 1 + 0.1)
    ax.set_xlabel("$n$th Excited State")

    ax.set_yscale("log")
    ax.set_ylabel("Energy (eV)")
    # ax.set_ylim(1e4, 1e6)

    ax.grid(axis="y", which="major", alpha=0.6)
    ax.grid(axis="y", which="minor", ls="dashed", alpha=0.6)
    ax.grid(axis="x", alpha=0.4)
    ax.set_axisbelow(True)

    ax.legend(handles=handles, loc="lower right")

    return fig, ax


fig, ax = multeng_vs_bs(
    etypes=etypes,
    dtype=dtype,
    qcl=qcl,
    num_states=num_states,
    labels=labels,
    colors=colors,
    markers=markers,
    markersizes=markersizes,
)

# <strong style="color:#8B1E3F">✨Autosavinator</strong><strong style="color:#DB4C40"> 3000✨</strong>

This scripts just automatically generates hundreds of plots like the ones above -- efficiency plots, histograms for mass and jet distribution and so on. It then saves them in a directory struture that hopefully makes sense.

In [ ]:
etypes = EVENT_CHOICES
dtypes = DATA_CHOICES
qcls = QCL_CHOICES
algs = ALG_CHOICES

# Different functions ran for brute force plots
bf_info = [
    [eng_vs_bs, "energy_vs_bitstring", "ENG VS BS", {}],
    [bf_mass_hist, "mass_2d_50t500", "MASS (50, 500)", {}],
    [bf_mass_hist, "mass_2d_0t250", "MASS (0, 250)", {"lims": (0, 250)}],
    [bf_njet_hist, "num_jets", "NUM JETS BOTH", {}],
    [bf_njet_hist, "num_jets0", "NUM JETS 0", {"bit": 0}],
    [bf_njet_hist, "num_jets1", "NUM JETS 1", {"bit": 1}],
]
# Same thing but for algorithms
alg_info = [
    [alg_mass_hist, "mass_2d_50t500", "MASS (50, 500)", {}],
    [alg_mass_hist, "mass_2d_0t250", "MASS (0, 250)", {"lims": (0, 250)}],
    [alg_njet_hist, "num_jets", "NUM JETS BOTH", {}],
    [alg_njet_hist, "num_jets0", "NUM JETS 0", {"bit": 0}],
    [alg_njet_hist, "num_jets1", "NUM JETS 1", {"bit": 1}],
]
# Non-alg, non-brute force plots/plots with both
extra_info = [
    [
        multalg_njet_hist,
        "mult_num_jets",
        "MNUM JETS BOTH",
        dict(
            algs=algs,
            depths=[8, 8, 8, 300],
            labels=[
                "QAOA",
                "ma-QAOA",
                "XQAOA",
                "FALQON",
                "Hemisphere method",
                "Hamiltonian limit",
            ],
            hatches=["x", "++", "**", ".O", "*-", "x|"],
            colors=["#76B041", "#2E282A", "#E4572E", "#17BEBB", "#FFC914", "#93867F"],
        ),
    ],
    [
        multalg_njet_hist,
        "mult_num_jets0",
        "MNUM JETS 0",
        dict(
            algs=algs,
            depths=[8, 8, 8, 300],
            labels=[
                "QAOA",
                "ma-QAOA",
                "XQAOA",
                "FALQON",
                "Hemisphere method",
                "Hamiltonian limit",
            ],
            hatches=["x", "++", "**", ".O", "*-", "x|"],
            colors=["#76B041", "#2E282A", "#E4572E", "#17BEBB", "#FFC914", "#93867F"],
            bit=0,
        ),
    ],
    [
        multalg_njet_hist,
        "mult_num_jets1",
        "MNUM JETS 1",
        dict(
            algs=algs,
            depths=[8, 8, 8, 300],
            labels=[
                "QAOA",
                "ma-QAOA",
                "XQAOA",
                "FALQON",
                "Hemisphere method",
                "Hamiltonian limit",
            ],
            hatches=["x", "++", "**", ".O", "*-", "x|"],
            colors=["#76B041", "#2E282A", "#E4572E", "#17BEBB", "#FFC914", "#93867F"],
            bit=1,
        ),
    ],
]
# Plots that use multiple algorithms and depend on invm
invm_info = [
    [
        eff_vs_numparams,
        "eff_vs_nparams",
        "EFF NPARAMS",
        dict(
            algs=["qaoa", "maqaoa", "xqaoa"],
            colors=["#3F88C5", "#AB2346", "#5F00BA"],
        ),
    ],
    [
        deltaE_vs_numparams,
        "deltaE_vs_nparams",
        "DE NPARAMS",
        dict(
            algs=["qaoa", "maqaoa", "xqaoa"],
            colors=["#3F88C5", "#AB2346", "#5F00BA"],
        ),
    ],
]
# Plots that use multiple etypes
multetype_info = [
    [
        multeng_vs_bs,
        "multenergy_vs_bitstring",
        "MENG VS BS",
        dict(
            etypes=etypes,
            num_states=16,
            colors=["#002642", "#840032", "#e59500"],
            markers=["o", "D", "*"],
            labels=[r"$t\bar{t}$", "$tW$", "$6$ jet"],
            markersizes=[7, 7, 15],
        ),
    ],
]

# Max character values for printing prettiness
labels = [x[2] for x in bf_info + extra_info + invm_info + alg_info]
lsmax = max([len(label) for label in labels]) + 2
asmax = max([len(alg) for alg in algs + ["BRUTE FORCE"]]) + 2
esmax = max([len(etype) for etype in etypes]) + 1
dsmax = max([len(dtype) for dtype in dtypes]) + 1
qsmax = max([len(qcl) for qcl in qcls])


def saver(plotter, save_dir, save_name, label, dtype, qcl, plotter_kwargs):
    """
    Saves plot created by function.

    Parameters:
    plotter - Function to call, must take the kwargs: 'dtype' and 'qcl' but
        can take other params. 'etype' is optional since some plots take multiple
        etypes.
    save_dir - Directory to save to
    save_name - File name to save
    label - Label name in square brackets when printed
    dtype - Data type
    qcl - Quad coefficient + lambda type as f"{quadcoeff}_{lmbda}"
    plotter_kwargs - The rest of kwargs for the function passed to 'plotter'
    """
    etype = plotter_kwargs.get("etype", "mult")
    save_path = save_dir / f"{save_name}.png"
    alg_name = f"[{plotter_kwargs.get('alg') or 'MISC'}]"
    label, elabel, dlabel = f"[{label}]", f"{etype},", f"{dtype},"
    combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {qcl:<{qsmax}})"
    if not save_path.exists():
        print(f"{alg_name:<{asmax}} {label:<{lsmax}} Saving {combo} to {save_path}...")
        fig, ax = plotter(dtype=dtype, qcl=qcl, **plotter_kwargs)
        fig.savefig(save_path)
        plt.close(fig)
    else:
        print(f"{alg_name:<{asmax}} {label:<{lsmax}} {combo} at {save_path} already exists, skipping...")


def deleter(name):
    num_deleted = 0
    for path in PLOTS_DIR.rglob(name):
        path.unlink()
        num_deleted += 1
    print(f"Deleted {num_deleted}...")


for dtype in dtypes:
    for qcl in qcls:
        save_dir = PLOTS_DIR / "multi_etype" / qcl
        if not save_dir.exists():
            save_dir.mkdir(parents=True)
        # Plots that aren't per etype
        for plotter, save_name, label, kwarg in multetype_info:
            # If all event files doesn't exist, don't even try
            for etype in kwarg["etypes"]:
                dont_run = False
                if not (EVT_DIR / f"{etype}_{dtype}.npy").exists():
                    print(f"No events for {etype}, {dtype}; skipping...")
                    dont_run = True
                    break
            if dont_run:
                continue

            saver(
                plotter=plotter,
                save_dir=save_dir,
                save_name=save_name,
                label=label,
                dtype=dtype,
                qcl=qcl,
                plotter_kwargs=kwarg,
            )

    for etype in etypes:
        # If event file doesn't exist, don't even try
        if not (EVT_DIR / f"{etype}_{dtype}.npy").exists():
            print(f"No events for {etype}, {dtype}; skipping...")
            continue

        # Cycle through qcl's
        for qcl in qcls:
            quadcoeff = qcl.split("_")[0]

            # If post data file doesn't exist, don't try
            if not (BF_DATA / "all_events" / f"bs_engs_{etype}_{dtype}.npz").exists():
                continue
            # Brute force stuff
            save_dir = PLOTS_DIR / "brute_force" / f"{etype}_{dtype}" / qcl
            if not save_dir.exists():
                save_dir.mkdir(parents=True)
            for plotter, save_name, label, kwarg in bf_info:
                kwarg |= {"etype": etype}
                saver(
                    plotter=plotter,
                    save_dir=save_dir,
                    save_name=save_name,
                    label=label,
                    dtype=dtype,
                    qcl=qcl,
                    plotter_kwargs=kwarg,
                )

            # All runs have used QA lambda, so just assume this and skip if the qcl isn't QA_QA
            if qcl.startswith("QA") and qcl != "QA_QA":
                continue

            for alg in algs:
                if not (ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz").exists():
                    continue

                # hard code for now
                depth = 300 if alg == "falqon" else 8
                try:
                    depth_dict[etype][dtype][quadcoeff][alg]
                except KeyError:
                    print(f"No data for algorithm {alg} for {etype}, {dtype}, {qcl}; skipping...")
                    continue

                save_dir = PLOTS_DIR / alg / f"{etype}_{dtype}" / qcl
                if not save_dir.exists():
                    save_dir.mkdir(parents=True)

                for plotter, save_name, label, kwarg in alg_info:
                    # for depth in depth_dict[etype][dtype][quadcoeff][alg]:
                    kwarg |= {"etype": etype, "alg": alg, "depth": depth}
                    saver(
                        plotter=plotter,
                        save_dir=save_dir,
                        save_name=save_name,
                        label=label,
                        dtype=dtype,
                        qcl=qcl,
                        plotter_kwargs=kwarg,
                    )

            # We assume for multi-alg plots that there is a kwarg "algs" so we check through
            # that for all necessary files
            save_dir = PLOTS_DIR / "multi_alg" / f"{etype}_{dtype}" / qcl
            if not save_dir.exists():
                save_dir.mkdir(parents=True)

            for plotter, save_name, label, kwarg in extra_info:
                # Make sure we have every algorithm file needed
                dont_run = False
                for alg in kwarg["algs"]:
                    if not (ALG_DATA / f"{alg}_{etype}_{dtype}_{qcl}.npz").exists():
                        dont_run = True
                        break
                if dont_run:
                    continue

                kwarg |= {"etype": etype}
                saver(
                    plotter=plotter,
                    save_dir=save_dir,
                    save_name=save_name,
                    label=label,
                    dtype=dtype,
                    qcl=qcl,
                    plotter_kwargs=kwarg,
                )

            for invm in INVMS[:-1] + ["all"]:
                invm_name = invm if invm == "all" else f"{invm:.2f}"
                save_dir = PLOTS_DIR / "multi_alg" / f"{etype}_{dtype}" / qcl
                if not save_dir.exists():
                    save_dir.mkdir(parents=True)

                for plotter, save_name, label, kwarg in invm_info:
                    # Make sure we have every algorithm file needed
                    dont_run = False
                    for alg in kwarg["algs"]:
                        if not (ALG_DATA / f"{alg}_{etype}_{dtype}_{qcl}.npz").exists():
                            dont_run = True
                            break
                    if dont_run:
                        continue

                    kwarg |= {"etype": etype, "invm_val": invm}
                    saver(
                        plotter=plotter,
                        save_dir=save_dir,
                        save_name=f"{save_name}_{invm_name}",
                        label=label,
                        dtype=dtype,
                        qcl=qcl,
                        plotter_kwargs=kwarg,
                    )

### EFFICIENCY PLOTS
for eff_type in ["correct", "min"]:
    save_dir = PLOTS_DIR / "efficiency" / ("ground_state" if eff_type == "min" else eff_type)
    if not save_dir.exists():
        save_dir.mkdir(parents=True)

    for etype, dtype, qcl in product(etypes, dtypes, qcls):
        if etype == "6jet":
            continue

        quadcoeff = qcl.split("_")[0]
        dont_run = False
        if quadcoeff == "QA" and qcl.split("_")[1] != "QA":
            continue
        for alg in algs:
            if not (ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz").exists():
                # if etype == "6jet" and dtype == "parton":
                #     continue
                dont_run = True
                break

        # Different algorithms on same plot
        if not dont_run:
            # Comparing all algorithms
            for do_max, do_hemi in product([True, False], [True, False]):
                if eff_type == "min" and (do_hemi or do_max):
                    continue
                col_lims = ["#ffba08", "#f48c06", "#dc2f02", "#9d0208", "#370617"]
                colors = multifader(col_lims, [0, 0.33, 0.66, 1])

                data_params = [
                    [etype, dtype, qcl, "qaoa", 8],
                    [etype, dtype, qcl, "maqaoa", 8],
                    [etype, dtype, qcl, "xqaoa", 8],
                    [etype, dtype, qcl, "falqon", 250],
                ]
                plot_params = [
                    [colors[0], "solid", 1, 3],
                    [colors[1], "solid", 1, 3],
                    [colors[2], "solid", 1, 3],
                    [colors[3], "solid", 1, 3],
                ]
                lines_params = [
                    [colors[0], "solid", 1, 3, "QAOA\n[$p=8$]"],
                    [colors[1], "solid", 1, 3, "ma-QAOA\n[$p=8$]"],
                    [colors[2], "solid", 1, 3, "XQAOA\n[$p=8$]"],
                    [colors[3], "solid", 1, 3, "FALQON\n[$p=250$]"],
                ]
                # # Haven't done this plot for FALQON yet, so remove it
                # if etype == "6jet":
                #     data_params = data_params[:-1]
                #     plot_params = plot_params[:-1]
                #     lines_params = lines_params[:-1]
                # Define what data for the Hemisphere method to use
                hemisphere_params = []
                if do_hemi:
                    hemisphere_params = [
                        [etype, dtype],
                        ["#8799a2", "solid", 1, 3, "Hemisphere\nmethod"],
                    ]
                # Define what maximum to plot
                max_params = []
                if do_max:
                    max_params = [[etype, dtype, qcl], ["#4d7EA8", "-", 0.3]]

                extra_str = f"{'_limit' if do_max else ''}{'_hemi' if do_hemi else ''}"
                save_path = save_dir / "alg_compare" / f"{etype}_{dtype}_{quadcoeff}{extra_str}.png"
                if not (save_dir / "alg_compare").exists():
                    (save_dir / "alg_compare").mkdir(parents=True)
                elabel, dlabel = f"{etype},", f"{dtype},"
                combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {qcl:<{qsmax}})"
                if not save_path.exists():
                    print(f"{'MISC':<{asmax}} {'EFF ALGS':<{lsmax}} Saving {combo} to {save_path}...")
                    fig, ax = eff_plot(
                        data_params=data_params,
                        plot_params=plot_params,
                        lines_params=lines_params,
                        patches_params=[],
                        max_params=max_params,
                        hemisphere_params=hemisphere_params,
                        correct_or_min=eff_type,
                    )
                    fig.savefig(save_path, bbox_inches="tight")
                    plt.close(fig)
                else:
                    print(f"{'MISC':<{asmax}} {'EFF ALGS':<{lsmax}} {combo} at {save_path} already exists, skipping...")

        # Different depths on same plot
        for alg in algs:
            if not (ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz").exists():
                continue
            # Comparing different depths of single algorithm
            for do_max, do_hemi in product([True, False], [True, False]):
                if eff_type == "min" and (do_hemi or do_max):
                    continue

                if alg == "falqon":
                    data_params = [
                        [etype, dtype, qcl, alg, 10],
                        [etype, dtype, qcl, alg, 25],
                        [etype, dtype, qcl, alg, 100],
                        [etype, dtype, qcl, alg, 250],
                        [etype, dtype, qcl, alg, 1000],
                    ]
                else:
                    data_params = [
                        [etype, dtype, qcl, alg, 1],
                        [etype, dtype, qcl, alg, 2],
                        [etype, dtype, qcl, alg, 5],
                        [etype, dtype, qcl, alg, 8],
                        [etype, dtype, qcl, alg, 25],
                    ]
                # Check that there is data for each depht
                if alg != "falqon":
                    good_params = []
                    max_depth = np.max(np.array(data_params)[:, -1].astype(int))
                    for data_param in data_params:
                        etype, dtype, qcl, alg, p = data_param
                        # If our choice for maximum depth isn't available, then just choose
                        # whatever IS the maximum depth (unless it's 8)
                        if p == max_depth and p not in depth_dict[etype][dtype][quadcoeff][alg] and p != 8:
                            data_param[-1] = max(depth_dict[etype][dtype][quadcoeff][alg])
                            good_params.append(data_param)
                        if p in depth_dict[etype][dtype][quadcoeff][alg]:
                            good_params.append(data_param)
                    data_params = good_params

                colors = ["#8ECAE6", "#219EBC", "#023047", "#FFB703", "#FB8500"]
                plot_params = [
                    [colors[0], "solid", 1, 3],
                    [colors[1], "solid", 1, 3],
                    [colors[2], "solid", 1, 3],
                    [colors[3], "solid", 1, 3],
                    [colors[4], "solid", 1, 3],
                ]
                depths = [10, 25, 100, 250, 1000] if alg == "falqon" else [1, 2, 5, 8, 25]
                lines_params = [
                    [colors[0], "solid", 1, 3, f"$p={depths[0]}$"],
                    [colors[1], "solid", 1, 3, f"$p={depths[1]}$"],
                    [colors[2], "solid", 1, 3, f"$p={depths[2]}$"],
                    [colors[3], "solid", 1, 3, f"$p={depths[3]}$"],
                    [colors[4], "solid", 1, 3, f"$p={depths[4]}$"],
                ]
                # Define what data for the Hemisphere method to use
                hemisphere_params = []
                if do_hemi:
                    hemisphere_params = [
                        [etype, dtype],
                        ["#8799a2", "solid", 1, 3, "Hemisphere\nmethod"],
                    ]
                # Define what maximum to plot
                max_params = []
                if do_max:
                    max_params = [[etype, dtype, qcl], ["#4d7EA8", "-", 0.3]]

                extra_str = f"{'_limit' if do_max else ''}{'_hemi' if do_hemi else ''}"
                save_path = save_dir / "depth_compare" / alg / f"{etype}_{dtype}_{quadcoeff}{extra_str}.png"
                if not (save_dir / "depth_compare" / alg).exists():
                    (save_dir / "depth_compare" / alg).mkdir(parents=True)
                elabel, dlabel = f"{etype},", f"{dtype},"
                combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {qcl:<{qsmax}})"
                if not save_path.exists():
                    print(f"{alg:<{asmax}} {'EFF P':<{lsmax}} Saving {combo} to {save_path}...")
                    fig, ax = eff_plot(
                        data_params=data_params,
                        plot_params=plot_params,
                        lines_params=lines_params,
                        patches_params=[],
                        max_params=max_params,
                        hemisphere_params=hemisphere_params,
                        correct_or_min=eff_type,
                    )
                    fig.savefig(save_path, bbox_inches="tight")
                    plt.close(fig)
                else:
                    print(f"{alg:<{asmax}} {'EFF P':<{lsmax}} {combo} at {save_path} already exists, skipping...")

    # Different hamiltonians on same plot
    for etype, dtype, alg in product(etypes, dtypes, algs):
        if etype == "6jet":
            continue

        depth = 250 if alg == "falqon" else 8
        # Comparing different depths of single algorithm
        for do_hemi in [True, False]:
            if eff_type == "min" and do_hemi:
                continue

            data_params = []
            plot_params = []
            lines_params = []
            colors = ["#005F73", "#9B2226", "#CA6702"]
            labels = ["$H_0$", "$H_1$", r"$H_1+\lambda H_1$"]
            for ind, quadcoeff in enumerate(["H0", "H1", "QA"]):
                if (ALG_DATA / f"{alg}_{etype}_{dtype}_{quadcoeff}.npz").exists():
                    data_params.append([etype, dtype, quadcoeff, alg, depth])
                    plot_params.append([colors[ind], "solid", 1, 3])
                    lines_params.append([colors[ind], "solid", 1, 3, labels[ind]])
            if not data_params:
                continue

            # Define what data for the Hemisphere method to use
            hemisphere_params = []
            if do_hemi:
                hemisphere_params = [
                    [etype, dtype],
                    ["#8799a2", "solid", 1, 3, "Hemisphere\nmethod"],
                ]

            extra_str = f"{'_limit' if do_max else ''}{'_hemi' if do_hemi else ''}"
            save_path = save_dir / "hamiltonian_compare" / alg / f"{etype}_{dtype}{extra_str}.png"
            if not (save_dir / "hamiltonian_compare" / alg).exists():
                (save_dir / "hamiltonian_compare" / alg).mkdir(parents=True)
            elabel, dlabel = f"{etype},", f"{dtype},"
            combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {qcl:<{qsmax}})"
            if not save_path.exists():
                print(f"{alg:<{asmax}} {'EFF QCL':<{lsmax}} Saving {combo} to {save_path}...")
                fig, ax = eff_plot(
                    data_params=data_params,
                    plot_params=plot_params,
                    lines_params=lines_params,
                    patches_params=[],
                    max_params=max_params,
                    hemisphere_params=hemisphere_params,
                    correct_or_min=eff_type,
                )
                fig.savefig(save_path, bbox_inches="tight")
                plt.close(fig)
            else:
                print(f"{alg:<{asmax}} {'EFF QCL':<{lsmax}} {combo} at {save_path} already exists, skipping...")

# Stuff for 6jet only
save_dir = PLOTS_DIR / "efficiency" / "6jet"
etype, dtype = "6jet", "parton"
if not save_dir.exists():
    save_dir.mkdir(parents=True)
# Comparing hamiltonians
for alg in algs:
    # Don't have FALQON data currently for 6 jets
    if alg == "falqon":
        continue

    colors = ["#005F73", "#9B2226", "#CA6702"]
    data_params = [
        ["6jet", "parton", "H0", alg, 8],
        ["6jet", "parton", "H1", alg, 8],
        ["6jet", "parton", "QA", alg, 8],
    ]
    plot_params = [
        [colors[0], "solid", 1, 3],
        [colors[1], "solid", 1, 3],
        [colors[2], "solid", 1, 3],
    ]
    lines_params = [
        [colors[0], "solid", 1, 3, "$H_0$"],
        [colors[1], "solid", 1, 3, "$H_1$"],
        [colors[2], "solid", 1, 3, r"$H_0+\lambda H_1$"],
    ]

    save_path = save_dir / "hamiltonian_compare" / f"{alg}_{etype}_{dtype}.png"
    if not (save_dir / "hamiltonian_compare").exists():
        (save_dir / "hamiltonian_compare").mkdir(parents=True)
    elabel, dlabel = f"{etype},", f"{dtype},"
    combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {'ALL':<{qsmax}})"
    if not save_path.exists():
        print(f"{'6JET':<{asmax}} {'EFF HAM':<{lsmax}} Saving {combo} to {save_path}...")
        fig, ax = eff_plot(
            data_params=data_params,
            plot_params=plot_params,
            lines_params=lines_params,
            patches_params=[],
            max_params=max_params,
            hemisphere_params=hemisphere_params,
            correct_or_min=eff_type,
        )
        fig.savefig(save_path, bbox_inches="tight")
        plt.close(fig)
    else:
        print(f"{'6JET':<{asmax}} {'EFF HAM':<{lsmax}} {combo} at {save_path} already exists, skipping...")
# Comparing algs
for qcl in qcls:
    quadcoeff = qcl.split("_")[0]
    if quadcoeff == "QA" and qcl.split("_")[1] != "QA":
        continue

    col_lims = ["#ffba08", "#f48c06", "#dc2f02", "#9d0208", "#370617"]
    colors = multifader(col_lims, [0, 0.33, 0.66, 1])
    data_params = [
        ["6jet", "parton", quadcoeff, "qaoa", 8],
        ["6jet", "parton", quadcoeff, "maqaoa", 8],
        ["6jet", "parton", quadcoeff, "xqaoa", 8],
    ]
    plot_params = [
        [colors[0], "solid", 1, 3],
        [colors[1], "solid", 1, 3],
        [colors[2], "solid", 1, 3],
    ]
    lines_params = [
        [colors[0], "solid", 1, 3, "QAOA\n[$p=8$]"],
        [colors[1], "solid", 1, 3, "ma-QAOA\n[$p=8$]"],
        [colors[2], "solid", 1, 3, "XQAOA\n[$p=8$]"],
    ]

    save_path = save_dir / "alg_compare" / f"{etype}_{dtype}_{quadcoeff}.png"
    if not (save_dir / "alg_compare").exists():
        (save_dir / "alg_compare").mkdir(parents=True)
    elabel, dlabel = f"{etype},", f"{dtype},"
    combo = f"({elabel:<{esmax}} {dlabel:<{dsmax}} {qcl:<{qsmax}})"
    if not save_path.exists():
        print(f"{'6JET':<{asmax}} {'EFF ALG':<{lsmax}} Saving {combo} to {save_path}...")
        fig, ax = eff_plot(
            data_params=data_params,
            plot_params=plot_params,
            lines_params=lines_params,
            patches_params=[],
            max_params=max_params,
            hemisphere_params=hemisphere_params,
            correct_or_min=eff_type,
        )
        fig.savefig(save_path, bbox_inches="tight")
        plt.close(fig)
    else:
        print(f"{'MISC':<{asmax}} {'EFF ALGS':<{lsmax}} {combo} at {save_path} already exists, skipping...")

# Delete all empty directories
print()
while True:
    num_deleted = 0
    for plot_dir in PLOTS_DIR.rglob("*/"):
        if next(plot_dir.iterdir(), None) is None:
            print(f"Deleting unused directory: {plot_dir}")
            num_deleted += 1
            plot_dir.rmdir()
    if num_deleted == 0:
        break

&nbsp;<hr style="border:5px solid #8d99ae">
# Misc Stuff

## Plotting Missing (Smeared) Data
I lost (most of) this data but had a plot of it. So, I took the number from the plots and copied them here to recreate the plots.

In [ ]:
# parton values
parton_qaoa = [0.056, 0.421, 0.700, 0.739, 0.742, 0.706]
parton_maqaoa = [0.039, 0.587, 0.906, 0.984, 0.997, 0.999]
parton_falqon = [0.028, 0.497, 0.863, 0.945, 0.970, 0.956]
parton_max = [0.108, 0.703, 0.939, 0.993, 1, 1]
# smeared values
smeared_qaoa = [0.043, 0.399, 0.659, 0.711, 0.752, 0.722]
smeared_maqaoa = [0.045, 0.508, 0.868, 0.969, 0.996, 0.999]
smeared_falqon = [0.032, 0.436, 0.770, 0.916, 0.920, 0.984]
smeared_max = [0.090, 0.623, 0.904, 0.980, 1, 1]
# hemisphere values
hemi_file = np.load(Path("data") / "hemisphere_efficiency.npz")
parton_hemi = hemi_file["Hemisphere_efficiency_parton"]
smeared_hemi = hemi_file["Hemisphere_efficiency_smeared"]


xs = np.array(list(zip(INVMS, INVMS))).reshape(-1)[1:-1]


def plot_it(ax, arr, color, ls, lw=3, fill=False):
    ys = np.array(list(zip(arr, arr))).reshape(-1)

    if fill:
        ax.fill_between(
            xs,
            ys,
            [1] * len(ys),
            color="gray",
            alpha=0.6,
            edgecolor="#444444",
            hatch="-",
        )
    else:
        ax.plot(xs, ys, color=color, ls=ls, lw=lw)


labels = ["1.00", "1.25", "1.50", "1.75", "2.00", "", "2.50", "", "3.00"]
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 18))
axes[0].set_box_aspect(1)
axes[0].set_xlim(1, 3)
axes[0].set_ylim(0, 1)
axes[0].tick_params(top=True, right=True, which="both")
axes[0].yaxis.set_major_locator(MultipleLocator(0.20))
axes[0].yaxis.set_minor_locator(MultipleLocator(0.05))
axes[0].grid(which="major", alpha=0.3)
axes[0].set_ylabel("Efficiency")
axes[0].set_xlabel("$m_{tt}/2m_t$")
axes[0].xaxis.set_major_locator(FixedLocator(xs))
plot_it(axes[0], parton_max, color="k", ls="solid", lw=3, fill=True)
plot_it(axes[0], parton_qaoa, color="#ffba08", ls="solid")
plot_it(axes[0], parton_maqaoa, color="#f48c06", ls="dashed")
plot_it(axes[0], parton_falqon, color="#dc2f02", ls="dotted")
plot_it(axes[0], parton_hemi, color="k", ls="dashdot")
axes[0].text(
    0.98,
    0.02,
    "Parton Events",
    fontsize=15,
    ha="right",
    va="bottom",
    transform=axes[0].transAxes,
)

axes[1].set_box_aspect(1)
axes[1].set_xlim(1, 3)
axes[1].set_ylim(0, 1)
axes[1].tick_params(top=True, right=True, which="both")
axes[1].yaxis.set_major_locator(MultipleLocator(0.20))
axes[1].yaxis.set_minor_locator(MultipleLocator(0.05))
axes[1].grid(which="major", alpha=0.3)
axes[1].set_ylabel("Efficiency")
axes[1].set_xlabel("$m_{tt}/2m_t$")
axes[1].xaxis.set_major_locator(FixedLocator(xs))
plot_it(axes[1], smeared_max, color="k", ls="solid", lw=4, fill=True)
plot_it(axes[1], smeared_qaoa, color="#ffba08", ls="solid")
plot_it(axes[1], smeared_maqaoa, color="#f48c06", ls="dashed")
plot_it(axes[1], smeared_falqon, color="#dc2f02", ls="dotted")
plot_it(axes[1], smeared_hemi, color="k", ls="dashdot")
axes[1].text(
    0.98,
    0.02,
    "Smeared Events",
    fontsize=15,
    ha="right",
    va="bottom",
    transform=axes[1].transAxes,
)

handles = [
    Patch(
        facecolor="gray",
        edgecolor="k",
        hatch="-",
        alpha=0.6,
        label="Hamiltonian\nLimit",
    ),
    Line2D([0], [0], color="#ffba08", ls="solid", lw=3, label="QAOA\n[$p=8$]"),
    Line2D([0], [0], color="#f48c06", ls="dashed", lw=3, label="maQAOA\n[$p=8$]"),
    Line2D([0], [0], color="#dc2f02", ls="dotted", lw=3, label="FALQON\n[$p=250$]"),
    Line2D([0], [0], color="k", ls="dashdot", lw=3, label="Hemisphere\nmethod"),
]

fig.legend(
    bbox_to_anchor=[1.04, 0.6],
    handles=handles,
    labelspacing=1.5,
    handlelength=3,
    handleheight=2.5,
)

## Analysis for shot-based data

In [ ]:
p = 5
steps = 1000
algs = ["maqaoa", "qaoa"]
delta = 10
used_shots = [
    10,
    25,
    50,
    100,
    250,
    500,
]

# Get data into useful format
placement_dicts = {}
for alg in algs:
    print(f"----------{alg.upper()}----------")
    placement_dict = defaultdict(dict)
    for invm in INVMS[:-1]:
        print(f"invm: {invm:.2f}")
        invm_dir = SHOT_DATA / f"shots_p{p}_steps{steps}_{alg}" / f"invm{invm:.2f}"
        shotss, indlos = np.array([re.findall(r"shots(\d+)_(\d+)to.*", shot_file.name)[0] for shot_file in invm_dir.glob("*")]).astype(int).T
        shotss = sorted(np.unique(shotss))

        print("  shots: ", end="")
        for shots in sorted(shotss):
            if shots not in used_shots + [0]:
                continue

            print(f"{shots}, ", end="")
            placement_dict[shots][invm] = []
            for indlo in indlos:
                data = np.load(invm_dir / f"shots{shots}_{indlo:0>4}to{indlo + delta - 1:0>4}.npz")
                placement_dict[shots][invm] += list(data["placements"])
        print()

    placement_dict = format_ddict(placement_dict)
    placement_dicts[alg] = placement_dict

In [48]:
# Useful function for adding sections to plot
xs = np.array(list(zip(INVMS, INVMS))).reshape(-1)[1:-1]


def plot_it(ax, arr, color, ls, lw=3, label=None, fill=False):
    ys = np.array(list(zip(arr, arr))).reshape(-1)

    if fill:
        ax.fill_between(
            xs,
            ys,
            [1] * len(ys),
            color="k",
            alpha=0.1,
            edgecolor="#000000",
            hatch="-",
        )
    else:
        ax.plot(xs, ys, color=color, ls=ls, lw=lw, label=label)

In [ ]:
fig, ax = plt.subplots(figsize=(9, 9))

colors = dict(
    zip(
        used_shots,
        multifader(
            [
                "#94d2bd",
                "#0a9396",
                "#005f73",
                "#ee9b00",
                "#ca6702",
                "#bb3e03",
                "#ae2012",
                "#9b2226",
            ],
            np.linspace(0, 1, len(used_shots)),
        ),
    )
)
ls_dict = {"maqaoa": "dashed", "qaoa": "dotted"}
for alg in algs:
    for shots in used_shots + [0]:
        effs = []
        for invm in INVMS[:-1]:
            placements = placement_dicts[alg][shots][invm]
            effs.append(np.sum(placements == 1) / len(placements))

        ls = ls_dict[alg]
        lw = 4 if shots == 0 else 2
        color = "#000000" if shots == 0 else colors[shots]
        fill = False
        plot_it(ax=ax, arr=effs, color=color, ls=ls, lw=lw, fill=fill)

ax.set_xlim(1, 3)
ax.set_ylim(0, 1)
ax.tick_params(top=True, right=True, which="both")
ax.grid(which="major", alpha=0.3)
ax.yaxis.set_major_locator(MultipleLocator(0.20))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))
ax.set_ylabel("Efficiency", fontsize=18)
ax.set_xlabel("$m_{tt}/2m_t$", fontsize=18)
ax.xaxis.set_major_locator(FixedLocator(xs))

handles = []
for shot in used_shots:
    handles.append(Patch(fc=colors[shot], label=shot))

ax.set_box_aspect(1)
ax.legend(handles=handles, title="Shots", title_fontsize=15, fontsize=15, loc="upper left")